# **Introduction**

# **Setup**

### Install dependencies

In [12]:
# Install missing dependencies
# !pip install -q torchinfo torchmetrics wandb
# !pip install torch
# !pip install torchvision
# !pip install numpy
# !pip install matplotlib


### Import important libraries

In [1]:
# Import required libraries/code
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from torchvision import transforms

from torchinfo import summary

import os
import glob
import wandb
import pickle

from torchvision.datasets import CIFAR100
from torch.utils.data import  random_split, DataLoader, Subset
import torch.nn.functional as F

### Build the directory for checkpoints

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

### Set device

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Data Preprocessing

In [4]:

# Define a class for handling CIFAR100 data
class CIFAR100Data:
    """
    A class used to represent the CIFAR100 dataset.

    ...

    Attributes
    ----------
    batch_size : int
        the number of samples that will be propagated through the network simultaneously
    original_train_set : torchvision.datasets.CIFAR100
        the original training set downloaded from CIFAR100
    original_test_set : torchvision.datasets.CIFAR100
        the original test set downloaded from CIFAR100
    train_set : torch.utils.data.Subset
        the training set after splitting the original training set
    validation_set : torch.utils.data.Subset
        the validation set after splitting the original training set
    test_set : torchvision.datasets.CIFAR100
        the test set, same as the original test set
    original_train_loader : torch.utils.data.DataLoader
        data loader for the original training set
    original_test_loader : torch.utils.data.DataLoader
        data loader for the original test set
    train_loader : torch.utils.data.DataLoader
        data loader for the training set
    validation_loader : torch.utils.data.DataLoader
        data loader for the validation set
    test_loader : torch.utils.data.DataLoader
        data loader for the test set

    Methods
    -------
    compute_mean_std(loader)
        Computes the mean and standard deviation of the images in the loader.
    download_data()
        Downloads the CIFAR100 dataset.
    split_data(original_train_set, validation_ratio=0.2)
        Splits the original training set into a training set and a validation set.
    compute_statistics(train_set)
        Computes the mean and standard deviation of the training set.
    apply_transforms(train_mean, train_std, is_validation_set_available = False)
        Defines and applies the transformations for the training set, validation set, and test set.
    save_data(data_loader, data_set, file_name: str)
        Saves the data loader to Google Drive.
    load_data(file_name: str)
        Loads the data loader from Google Drive.
    create_and_save_data_loaders(train_set, test_set, validation_set=None)
        Creates data loaders for the training, validation, and test sets and saves them to Google Drive.
    prepare_data(validation_ratio = None)
        Prepares the data by downloading it, splitting it, computing statistics, applying transforms, and creating and saving data loaders.
    train_valid_test(validation_ratio=0.2)
        Loads or prepares the data loaders for the training, validation, and test sets and returns them.
    train_test()
        Loads or prepares the data loaders for the original training and test sets and returns them.
    iid_shards(num_shards=2)
        Loads or prepares the data loaders for the shards of the original training set and returns them.
    """
    def __init__(self, batch_size=64):
        """
        Initialize the CIFAR100Data object with the given batch size.

        Parameters:
        batch_size (int): The size of the batches for the data loaders.
        """
        self.batch_size = batch_size
        self.original_train_set = None
        self.original_test_set = None
        self.train_set = None
        self.validation_set = None
        self.test_set = None

        self.original_train_loader = None
        self.original_test_loader = None
        self.train_loader = None
        self.validation_loader = None
        self.test_loader = None

    def compute_mean_std(self, loader):
        """
        Compute the mean and standard deviation of the images in the loader.

        Parameters:
        loader (DataLoader): The DataLoader object containing the image data.

        Returns:
        mean (Tensor): The mean of the images.
        std (Tensor): The standard deviation of the images.
        """
        channels_sum, channels_squared_sum, num_batches = 0, 0, 0
        for data, _ in loader:
            channels_sum += torch.mean(data, dim=[0, 2, 3])
            channels_squared_sum += torch.mean(data**2, dim=[0, 2, 3])
            num_batches += 1
        mean = channels_sum / num_batches
        std = torch.sqrt((channels_squared_sum / num_batches) - mean**2)
        return mean, std

    def download_data(self):
        """
        Download the CIFAR100 dataset and store it in instance variables.
        """
        self.original_train_set = CIFAR100(root='./data', train=True, download=True, transform=transforms.ToTensor())
        self.original_test_set = CIFAR100(root='./data', train=False, download=True, transform=transforms.ToTensor())

    def split_data(self, original_train_set, validation_ratio=0.2):
        """
        Split the original training set into a training set and a validation set.

        Parameters:
        original_train_set (Dataset): The original training set.
        validation_ratio (float): The ratio of the original training set to use for validation.

        Returns:
        train_set (Subset): The new training set.
        validation_set (Subset): The new validation set.
        """
        train_len = int(len(original_train_set) * (1 - validation_ratio))
        val_len = len(original_train_set) - train_len
        train_set, validation_set = random_split(original_train_set, [train_len, val_len])

        return train_set, validation_set

    def compute_statistics(self, train_set):
        """
        Compute the mean and standard deviation of the train set.

        Parameters:
        train_set (Dataset/Subset): The training set.

        Returns:
        train_mean (Tensor): The mean of the training set.
        train_std (Tensor): The standard deviation of the training set.
        """
        trainloader_tmp = DataLoader(train_set, batch_size=self.batch_size, shuffle=True)
        train_mean, train_std = self.compute_mean_std(trainloader_tmp)

        return train_mean, train_std

    def apply_transforms(self, train_mean, train_std, is_validation_set_available = False):
        """
        Define the transformations for the training set, validation set, and test set
        and apply them to the datasets.

        Parameters:
        train_mean (Tensor): The mean of the training set.
        train_std (Tensor): The standard deviation of the training set.
        is_validation_set_available (bool): Whether a validation set is available.
        """

        # Transformations for the training set
        train_transforms = transforms.Compose([
            transforms.RandomCrop(32, padding=4, padding_mode='reflect'),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(train_mean.tolist(), train_std.tolist())
        ])

        # Transformations for the validation and test sets
        test_val_transforms = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(train_mean.tolist(), train_std.tolist())
        ])

        # Apply the transformations to the datasets
        if is_validation_set_available:
            self.train_set.transform = train_transforms
            self.validation_set.transform = test_val_transforms
            self.test_set.transform = test_val_transforms
        else:
            self.original_train_set.transform = train_transforms
            self.original_test_set.transform = test_val_transforms

    def save_data(self, data_loader, file_name: str):
        """
        Save the given data loader and data set to Google Drive.

        Parameters:
        data_loader (DataLoader): The data loader to save.
        file_name (str): The name of the file to save the data loader and data set to.
        """
        # Check if the directory exists, if not, create it
        # if not os.path.exists('/content/gdrive/MyDrive/data/data_loaders/'):
        #     os.makedirs('/content/gdrive/MyDrive/data/data_loaders/')

        # Open each file in write-binary mode on Google Drive and dump (pickle) the data loader into it
        # with open(f'/content/gdrive/MyDrive/data/data_loaders/{self.batch_size}_{file_name}_loader.pkl', 'wb') as f:
        #     pickle.dump(data_loader, f)
        if not os.path.exists(f'./data/data_loaders/{self.batch_size}/'):
            os.makedirs(f'./data/data_loaders/{self.batch_size}/')
        
        open(f'./data/data_loaders/{self.batch_size}/{file_name}_loader.pkl', 'wb').write(pickle.dumps(data_loader))

    def load_data(self, file_name: str):
        """
        Load a data loader from Google Drive.

        Parameters:
        file_name (str): The name of the file to load the data loader from.

        Returns:
        data_loader (DataLoader): The loaded data loader, or None if the file does not exist.
        """
        # Check if the file exists
        # if os.path.exists(f'/content/gdrive/MyDrive/data/data_loaders/{self.batch_size}_{file_name}_loader.pkl'):
        #     # If it exists, open the file in read-binary mode and load (unpickle) the data loader from it
        #     with open(f'/content/gdrive/MyDrive/data/data_loaders/{self.batch_size}_{file_name}_loader.pkl', 'rb') as f:
        #         return pickle.load(f)
        # else:
        #     return None
        if os.path.exists(f'./data/data_loaders/{self.batch_size}/{file_name}_loader.pkl'):
            return pickle.loads(open(f'./data/data_loaders/{self.batch_size}/{file_name}_loader.pkl', 'rb').read())
        else:
            return None

    def create_and_save_data_loaders(self, train_set, test_set, train_name: str, test_name: str, validation_set=None):
        """
        Create data loaders for the training, validation, and test sets and save them to Google Drive.

        Parameters:
        train_set (Subset): The training set.
        test_set (Subset): The test set.
        validation_set (Subset, optional): The validation set.

        Returns:
        train_loader (DataLoader): The data loader for the training set.
        validation_loader (DataLoader): The data loader for the validation set, if it exists.
        test_loader (DataLoader): The data loader for the test set.
        """
        train_loader = DataLoader(train_set, batch_size=self.batch_size, shuffle=True, num_workers =8)
        test_loader = DataLoader(test_set, batch_size=self.batch_size, shuffle=False, num_workers=8)

        # Save the newly created data loaders to Google Drive
        self.save_data(train_loader, train_name)
        self.save_data(test_loader, test_name)

        if validation_set is not None:
            validation_loader = DataLoader(validation_set, batch_size=self.batch_size, shuffle=False, num_workers=8)
            self.save_data(validation_loader, 'validation')
            return train_loader, validation_loader, test_loader

        return train_loader, test_loader

    def prepare_data(self, validation_ratio = None):
        """
        Prepare the data by downloading it, splitting it into training, validation, and test sets,
        computing statistics, applying transformations, and creating and saving data loaders.

        Parameters:
        validation_ratio (float, optional): The ratio of the original training set to use for validation.
        """
        try:
            self.download_data()
        except IOError:
            print("Error downloading data")
            return

        if validation_ratio is not None:
            self.train_set, self.validation_set = self.split_data(self.original_train_set, validation_ratio)
            if self.validation_set is None:
                print("Validation set is not available")
                return
            self.test_set = self.original_test_set
            train_mean, train_std = self.compute_statistics(self.train_set)
            self.apply_transforms(train_mean, train_std, is_validation_set_available = True)

            # Create and save data loaders
            self.train_loader, self.validation_loader, self.test_loader = self.create_and_save_data_loaders(self.train_set, self.test_set, 'train', 'test', self.validation_set)

        else:
            train_mean, train_std = self.compute_statistics(self.original_train_set)
            self.apply_transforms(train_mean, train_std)

            # Create and save data loaders
            self.original_train_loader, self.original_test_loader = self.create_and_save_data_loaders(self.original_train_set, self.original_test_set, 'original_train', 'original_test')

    def train_valid_test(self, validation_ratio=0.2):
        """
        Load the training, validation, and test data loaders from Google Drive, or prepare the data if they do not exist.

        Parameters:
        validation_ratio (float): The ratio of the original training set to use for validation.

        Returns:
        train_loader (DataLoader): The data loader for the training set.
        validation_loader (DataLoader): The data loader for the validation set.
        test_loader (DataLoader): The data loader for the test set.
        """
        self.train_loader = self.load_data('train')
        self.validation_loader = self.load_data('validation')
        self.test_loader = self.load_data('test')

        if self.train_loader is None or self.validation_loader is None or self.test_loader is None:
            self.prepare_data(validation_ratio)

        # Return the data loaders
        return self.train_loader, self.validation_loader, self.test_loader

    def train_test(self):
        """
        Load the original training and test data loaders from Google Drive, or prepare the data if they do not exist.

        Returns:
        original_train_loader (DataLoader): The data loader for the original training set.
        original_test_loader (DataLoader): The data loader for the original test set.
        """
        self.original_train_loader = self.load_data('original_train')
        self.original_test_loader = self.load_data('original_test')

        if self.original_train_loader is None or self.original_test_loader is None:
            self.prepare_data()

        # Return the data loaders
        return self.original_train_loader, self.original_test_loader

    def iid_shards(self, num_shards=2):
        """
        Create or load independent and identically distributed (IID) shards of the original training set.

        Parameters:
        num_shards (int): The number of shards to create.

        Returns:
        shard_loaders (list of DataLoader): The data loaders for the shards.
        """
        # Try to load the shard datasets and their corresponding data loaders from Google Drive
        shard_loaders = []
        for i in range(num_shards):
            shard_loader = self.load_data(f'iid_sharding/{num_shards}_chunk_{i+1}')
            if shard_loader is None:
                break
            shard_loaders.append(shard_loader)

        # If all shard data loaders were successfully loaded, return them
        if len(shard_loaders) == num_shards:
            return shard_loaders

        # If not all shard data loaders were successfully loaded, create them
        if self.original_train_set is None:
            self.download_data()
            train_mean, train_std = self.compute_statistics(self.original_train_set)
            self.apply_transforms(train_mean, train_std)

        # Shuffle the indices
        indices = torch.randperm(len(self.original_train_set))

        # Split the indices into K chunks
        shard_size = len(indices) // num_shards
        shards = [indices[i*shard_size:(i+1)*shard_size] for i in range(num_shards)]

        # Create subsets for each shard
        shard_datasets = [Subset(self.original_train_set, shard) for shard in shards]

        # Create data loaders for each shard
        shard_loaders = [DataLoader(shard_dataset, batch_size=self.batch_size, shuffle=True) for shard_dataset in shard_datasets]

        # Save each shard dataset and its corresponding data loader
        for i, shard_loader in enumerate(shard_loaders):
            self.save_data(shard_loader, f'iid_sharding_{num_shards}_chunk_{i+1}')

        return shard_loaders


In [5]:
data = CIFAR100Data()
train_loader, validation_loader, test_loader = data.train_valid_test(validation_ratio=0.2)
original_train_loader, original_test_loader = data.train_test()

### Define the model architecture

In [6]:
# Define the model architecture
# Check if it is LeNet-5 or similar to LeNet-5 we want similar.
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=0) # 28x28
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=0) # 10x10
        # TODO: be nazaram inja 2 ta pool tarif konim behtar bashe
        self.pool = nn.MaxPool2d(2, 2) # 14x14 for conv1 and 5x5 for conv2
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(5 * 5 * 64, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x, indexing=None):
        intermediate_outputs = {}

        # Convolutional and pooling layers
        x = F.relu(self.conv1(x))
        if indexing == 'conv1':
            return x
        intermediate_outputs['conv1'] = x

        x = self.pool(x)
        if indexing == 'pool1':
            return x
        intermediate_outputs['pool1'] = x

        x = F.relu(self.conv2(x))
        if indexing == 'conv2':
            return x
        intermediate_outputs['conv2'] = x

        x = self.pool(x)
        if indexing == 'pool2':
            return x
        intermediate_outputs['pool2'] = x

        # Flatten layer
        x = self.flatten(x)
        if indexing == 'flatten':
            return x
        intermediate_outputs['flatten'] = x

        # Fully connected layers
        x = F.relu(self.fc1(x))
        if indexing == 'fc1':
            return x
        intermediate_outputs['fc1'] = x

        x = F.relu(self.fc2(x))
        if indexing == 'fc2':
            return x
        intermediate_outputs['fc2'] = x

        x = self.fc3(x)
        if indexing == 'fc3':
            return x
        intermediate_outputs['fc3'] = x

        # If no indexing, return final output
        return x if indexing is None else intermediate_outputs.get(indexing, x)

In [7]:
# Define loss function
criterion = nn.CrossEntropyLoss()

### Define some basic functions for train and test the model

In [8]:
def train(model, dataloader, optimizer, loss_fn, accumulation_steps=1, device=device, is_wandb=False):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    optimizer.zero_grad()  # Initialize gradients to zero at the start of each epoch

    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        loss.backward()  # Backpropagation

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # Gradient accumulation
        if (i+1) % accumulation_steps == 0 or i+1 == len(dataloader):
            optimizer.step()  # Update model parameters
            optimizer.zero_grad()  # Reset gradients to zero

    train_loss = running_loss / len(dataloader)
    train_accuracy = 100. * correct / total

    if is_wandb:
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_accuracy})

    return train_loss, train_accuracy

In [9]:
def test(model, dataloader, loss_fn, device = device, is_wandb= False):

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            if is_wandb:
              # Log the loss and accuracy values at each step
              wandb.log({
                  'Test Loss': test_loss / (batch_idx + 1),
                  'Test Accuracy': 100 * correct / total
              })

    test_loss = test_loss / len(dataloader)
    test_accuracy = 100. * correct / total

    return test_loss, test_accuracy

In [10]:
def save_checkpoint(state, epoch, batch_size, optimizer_name, hyperparameters):
    # dir_path = f"/content/gdrive/MyDrive/{optimizer_name}/{batch_size}/"
    dir_path = f"./train/{optimizer_name}/{batch_size}/"
    for key, value in hyperparameters.items():
        dir_path = os.path.join(dir_path, f"{key}_{value}/")
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    path = os.path.join(dir_path, f"epoch_{epoch:03}.pt")
    torch.save(state, path)

    # Get list of all files
    list_of_files = glob.glob(os.path.join(dir_path, f"epoch_*.pt"))
    # Sort files by creation time
    list_of_files.sort(key=os.path.getctime)
    # If there are more than 2 files, delete the second last one
    if len(list_of_files) > 1:
        os.remove(list_of_files[-2])

def load_checkpoint(optimizer_name, batch_size, hyperparameters):
    # dir_path = f"/content/gdrive/MyDrive/{optimizer_name}/{batch_size}/"
    dir_path = f"./train/{optimizer_name}/{batch_size}/"
    for key, value in hyperparameters.items():
        dir_path = os.path.join(dir_path, f"{key}_{value}/")
    list_of_files = glob.glob(os.path.join(dir_path, f"epoch_*.pt")) # * means all if need specific format then *.csv
    if not list_of_files:  # I'm using glob which can return an empty list
        return None
    latest_file = max(list_of_files, key=os.path.getctime)
    if os.path.isfile(latest_file):
        return torch.load(latest_file)
    return None

In [11]:
def run_training(
    num_epochs,
    model,
    trainloader,
    validationloader,
    testloader,
    optimizer,
    scheduler,
    loss_fn,
    device,
    optimizer_name: str,
    accumulation_steps=1,
    hyperparameters=None,
    is_wandb = False,
    n_epochs_stop = None
  ):

  best_accuracy = 0
  epochs_no_improve = 0
  n_epochs_stop = n_epochs_stop  # number of epochs to wait before stopping

  start_epoch = 0
  run_id = None
  run_name = None
  # Load checkpoint if available
  checkpoint = load_checkpoint(optimizer_name, trainloader.batch_size, hyperparameters)
  if checkpoint is not None:
      model.load_state_dict(checkpoint['model_state_dict'])
      optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
      scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
      start_epoch = checkpoint['epoch'] + 1
      run_id = checkpoint.get('wandb_run_id', None)
      run_name = checkpoint.get('wandb_run_name', None)

  else:
    run_name = " ".join([f"{key}={value}" for key, value in hyperparameters.items()])

  if is_wandb:
    # Initialize a wandb run with the given hyperparameters
    wandb.init(id=run_id, name=run_name, project=f'cifar100-training-mldl2024-baseline-{optimizer_name}',
                   config=hyperparameters if hyperparameters is not None else {},
                   resume="allow", reinit=True)

  for epoch in range(start_epoch, num_epochs):
      # Call the training function for each epoch
      train_loss, train_acc = train(model, trainloader, optimizer, loss_fn, accumulation_steps, device, is_wandb=is_wandb)
      print(f'[{epoch+1}/{num_epochs}]: Training Loss: {train_loss}, Training Accuracy: {train_acc}')
      scheduler.step() # Update learning rate based on scheduler

      # Save checkpoint
      save_checkpoint({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict': scheduler.state_dict(),
                    'loss': criterion,
                    'wandb_run_id': wandb.run.id if is_wandb else None,
                    'wandb_run_name': wandb.run.name if is_wandb else None,
                    }, epoch, trainloader.batch_size, optimizer_name, hyperparameters)

      if validationloader is not None:
        val_loss, val_acc = test(model, validationloader, criterion)
        print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')


        if n_epochs_stop is not None:
          if val_acc > best_accuracy:
                best_accuracy = val_acc
                epochs_no_improve = 0
          else:
              epochs_no_improve += 1
              if epochs_no_improve == n_epochs_stop:
                  print('Early stopping!')
                  break

  print('*'*70)
  test_loss, test_acc = test(model, testloader, criterion, is_wandb = is_wandb)
  print(f'Test Loss: {test_loss}, Test Accuracy: {test_acc}')


  # Finish the wandb run after all epochs
  wandb.finish()

# **Centeralised baseline**

In [70]:

learning_rates = [1e-03, 1e-02]
weight_decays = [1e-04, 1e-03, 4e-04]


#### SGDM (Stochastic Gradient Descent with Momentum)

##### Hyperparameter Tuning

In [60]:
num_epochs = 150

for lr in learning_rates:
  for wd in weight_decays:

    print('='*50)
    print(f'Hyperparameter with lr:{lr} and wd:{wd}')
    print('='*50)

    hyperparameters = {'learning_rate': lr,
                       'weight_decay' : wd
                       }
    # Load the model
    model = LeNet5().to(device)

    # Optimizer and scheduler setup
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


    # Training
    run_training(num_epochs, model, train_loader, validation_loader, test_loader, optimizer, scheduler, criterion, device, 'SGDM-HyperParameterTuning', hyperparameters=hyperparameters, is_wandb = True, n_epochs_stop = 10)



Hyperparameter with lr:0.001 and wd:0.0001


[10/150]: Training Loss: 3.7955315692901612, Training Accuracy: 12.3825
Validation Loss: 3.7704276780413974, Validation Accuracy: 12.97
[11/150]: Training Loss: 3.693796951675415, Training Accuracy: 14.2675
Validation Loss: 3.6661312990127857, Validation Accuracy: 14.76
[12/150]: Training Loss: 3.5919596023559572, Training Accuracy: 16.0125
Validation Loss: 3.5912161389733575, Validation Accuracy: 16.26
[13/150]: Training Loss: 3.5116733703613283, Training Accuracy: 17.3775
Validation Loss: 3.5196323546634356, Validation Accuracy: 16.95
[14/150]: Training Loss: 3.4402521675109865, Training Accuracy: 18.835
Validation Loss: 3.4998342930131656, Validation Accuracy: 17.78
[15/150]: Training Loss: 3.374294019317627, Training Accuracy: 19.835
Validation Loss: 3.412280123704558, Validation Accuracy: 19.05
[16/150]: Training Loss: 3.3126377914428713, Training Accuracy: 20.9225
Validation Loss: 3.3567575497232425, Validation Accuracy: 20.21
[17/150]: Training Loss: 3.264142190551758, Training 

Test Accuracy,██▁▂▂▂▂▁▁▁▁▂▂▂▃▃▃▃▃▂▂▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
Test Loss,▁▄▆▇▇▇███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Train Accuracy,▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
Train Loss,██▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁
Test Accuracy,12.53
Test Loss,12.75723
Train Accuracy,59.32
Train Loss,1.48869


Hyperparameter with lr:0.001 and wd:0.001


[1/150]: Training Loss: 4.605354942321777, Training Accuracy: 1.0075
Validation Loss: 4.604883777108162, Validation Accuracy: 0.9
[2/150]: Training Loss: 4.602973918151855, Training Accuracy: 1.0475
Validation Loss: 4.6016397293965525, Validation Accuracy: 0.9
[3/150]: Training Loss: 4.596875128936768, Training Accuracy: 1.3425
Validation Loss: 4.590152442834939, Validation Accuracy: 2.52
[4/150]: Training Loss: 4.556022624969483, Training Accuracy: 2.9175
Validation Loss: 4.472016604842653, Validation Accuracy: 3.55
[5/150]: Training Loss: 4.280242394256592, Training Accuracy: 4.6625
Validation Loss: 4.20257385673037, Validation Accuracy: 5.95
[6/150]: Training Loss: 4.122820203781128, Training Accuracy: 6.5925
Validation Loss: 4.098185727550725, Validation Accuracy: 6.72
[7/150]: Training Loss: 4.0448949375152585, Training Accuracy: 7.655
Validation Loss: 4.033748802865387, Validation Accuracy: 8.54
[8/150]: Training Loss: 3.977578921508789, Training Accuracy: 9.06
Validation Loss: 3

...

[1/150]: Training Loss: 4.452998904800415, Training Accuracy: 2.615
Validation Loss: 4.1346032179085315, Validation Accuracy: 5.28
[2/150]: Training Loss: 3.9527258255004885, Training Accuracy: 8.88
Validation Loss: 3.7435999432946465, Validation Accuracy: 11.55
[3/150]: Training Loss: 3.6381911922454835, Training Accuracy: 14.3725
Validation Loss: 3.50478922181828, Validation Accuracy: 15.58
[4/150]: Training Loss: 3.3785652015686036, Training Accuracy: 18.865
Validation Loss: 3.3451961089091697, Validation Accuracy: 19.0
[5/150]: Training Loss: 3.17810930519104, Training Accuracy: 22.375
Validation Loss: 3.1223374703887163, Validation Accuracy: 23.6
[6/150]: Training Loss: 3.0013810291290284, Training Accuracy: 25.565
Validation Loss: 3.015145599462424, Validation Accuracy: 25.56
[7/150]: Training Loss: 2.8544215950012206, Training Accuracy: 28.7175
Validation Loss: 2.917889818264421, Validation Accuracy: 28.04
[8/150]: Training Loss: 2.6971351528167724, Training Accuracy: 31.3825
Va

Test Accuracy,▄█▄▅▃▄▄▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,▁▂▆▄▃▅▆▇▇▆▆▇▇▇███▇▇▇██▇▇█▇██████████████
Train Accuracy,▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇███
Train Loss,█▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁▁
Test Accuracy,18.45
Test Loss,17.25387
Train Accuracy,77.925
Train Loss,0.72145


Hyperparameter with lr:0.01 and wd:0.0004


[1/150]: Training Loss: 4.436210793685913, Training Accuracy: 2.8025
Validation Loss: 4.1512272403498365, Validation Accuracy: 5.38
[2/150]: Training Loss: 3.943741687011719, Training Accuracy: 9.175
Validation Loss: 3.791590502307673, Validation Accuracy: 11.54
[3/150]: Training Loss: 3.6135140613555907, Training Accuracy: 14.57
Validation Loss: 3.473675949558331, Validation Accuracy: 17.5
[4/150]: Training Loss: 3.3465395004272462, Training Accuracy: 19.22
Validation Loss: 3.3301091406755385, Validation Accuracy: 19.19
[5/150]: Training Loss: 3.1310076709747316, Training Accuracy: 22.94
Validation Loss: 3.0678081117617855, Validation Accuracy: 24.65
[6/150]: Training Loss: 2.9402838905334474, Training Accuracy: 26.4875
Validation Loss: 2.9766739310732313, Validation Accuracy: 25.68
[7/150]: Training Loss: 2.7704892574310302, Training Accuracy: 29.855
Validation Loss: 2.8771827342403924, Validation Accuracy: 28.79
[8/150]: Training Loss: 2.6034393648147582, Training Accuracy: 33.84
Va

Test Accuracy,█▆▁▅▆▇█▆▆▆▇▇▆▇▇▇▇▇████████▇██▇▇▇▇███████
Test Loss,▁▄▆▅▅▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████
Train Accuracy,▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇███
Train Loss,█▇▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▁▁▁▁
Test Accuracy,17.1
Test Loss,24.73122
Train Accuracy,80.77
Train Loss,0.6179


##### Training Using Best Hyperparameters Set

In [64]:
num_epochs = 150
lr = 1e-02
wd = 1e-03

hyperparameters = {'learning_rate': lr,
                  'weight_decay' : wd}

# Load the model
model_0 = LeNet5().to(device)

# Optimizer and scheduler setup
optimizer_0 = torch.optim.SGD(model_0.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_0, T_max=num_epochs)


# Training
run_training(num_epochs, model_0, original_train_loader, original_test_loader, original_test_loader, optimizer_0, scheduler, criterion, device, optimizer_name='SGDM', hyperparameters=hyperparameters, is_wandb=True)


[1/150]: Training Loss: 4.089274020146226, Training Accuracy: 7.336
Validation Loss: 3.675565988394865, Validation Accuracy: 11.94
[2/150]: Training Loss: 3.4894355224526445, Training Accuracy: 16.328
Validation Loss: 3.239326788361665, Validation Accuracy: 21.43
[3/150]: Training Loss: 3.1558719864281852, Training Accuracy: 21.82
Validation Loss: 2.9393957374961515, Validation Accuracy: 26.07
[4/150]: Training Loss: 2.9186559430778485, Training Accuracy: 26.722
Validation Loss: 2.7213429053118277, Validation Accuracy: 30.34
[5/150]: Training Loss: 2.7546427048685604, Training Accuracy: 29.83
Validation Loss: 2.6445619574018346, Validation Accuracy: 32.87
[6/150]: Training Loss: 2.6314107673552334, Training Accuracy: 32.316
Validation Loss: 2.463979342940507, Validation Accuracy: 35.97
[7/150]: Training Loss: 2.525907842399519, Training Accuracy: 34.748
Validation Loss: 2.402360181899587, Validation Accuracy: 37.43
[8/150]: Training Loss: 2.4479937202790203, Training Accuracy: 36.512
V

Test Accuracy,▆█▄▃▄▃▂▁▂▂▂▂▁▁▁▁▂▂▂▁▁▁▁▂▁▂▁▂▂▂▂▁▂▂▂▂▂▂▂▂
Test Loss,█▁▄▅▅▅▆▇▅▆▅▄▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▃▄
Train Accuracy,▁▂▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇███████
Train Loss,█▇▆▅▅▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
Test Accuracy,57.55
Test Loss,1.89869
Train Accuracy,87.314
Train Loss,0.44531


#### AdamW (Adam with Weight Decay)

##### Hyperparameter Tuning

In [71]:
num_epochs = 150

for lr in learning_rates:
  for wd in weight_decays:

    print('='*50)
    print(f'Hyperparameter with lr:{lr} and wd:{wd}')
    print('='*50)

    hyperparameters = {'learning_rate': lr,
                       'weight_decay' : wd
                       }
    # Load the model
    model = LeNet5().to(device)

    # Optimizer and scheduler setup
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


    # Training
    run_training(num_epochs, model, train_loader, validation_loader, test_loader, optimizer, scheduler, criterion, device, 'AdamW-HyperParameterTuning', hyperparameters=hyperparameters, is_wandb = True, n_epochs_stop = 10)



Hyperparameter with lr:0.001 and wd:0.0001


Train Accuracy,▁
Train Loss,▁
Train Accuracy,7.7625
Train Loss,4.01706


[1/150]: Training Loss: 4.032126424407959, Training Accuracy: 7.645
Validation Loss: 3.633875617555752, Validation Accuracy: 13.81
[2/150]: Training Loss: 3.4117616390228274, Training Accuracy: 17.9375
Validation Loss: 3.291252329091358, Validation Accuracy: 19.57
[3/150]: Training Loss: 3.0980306770324706, Training Accuracy: 23.7325
Validation Loss: 3.0133402590539045, Validation Accuracy: 25.49
[4/150]: Training Loss: 2.875119793319702, Training Accuracy: 27.905
Validation Loss: 2.9483105938905365, Validation Accuracy: 26.77
[5/150]: Training Loss: 2.6911334384918213, Training Accuracy: 31.615
Validation Loss: 2.851942961383018, Validation Accuracy: 28.77
[6/150]: Training Loss: 2.5383362628936768, Training Accuracy: 34.6425
Validation Loss: 2.8010447116414454, Validation Accuracy: 30.38
[7/150]: Training Loss: 2.3951899276733397, Training Accuracy: 37.7375
Validation Loss: 2.772052654035532, Validation Accuracy: 31.08
[8/150]: Training Loss: 2.2677825828552245, Training Accuracy: 40

Test Accuracy,█▇▃▃▁▁▂▂▂▂▃▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Test Loss,▁▆▇▅████████▇█▇▇██▇█▇███▇███▇▇█▇███▇▇▇▇▇
Train Accuracy,▁▂▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇███
Train Loss,█▇▆▅▅▄▄▄▄▃▃▃▂▂▂▂▂▁▁▁
Test Accuracy,3.39
Test Loss,198.79228
Train Accuracy,68.56
Train Loss,1.0573


Hyperparameter with lr:0.001 and wd:0.001


...

[1/150]: Training Loss: 4.625637398529053, Training Accuracy: 0.885
Validation Loss: 4.60909015509733, Validation Accuracy: 1.06
[2/150]: Training Loss: 4.608977200317383, Training Accuracy: 0.94
Validation Loss: 4.609363568056921, Validation Accuracy: 0.92
[3/150]: Training Loss: 4.608765270996094, Training Accuracy: 0.9275
Validation Loss: 4.610460181145152, Validation Accuracy: 0.9
[4/150]: Training Loss: 4.609055269622803, Training Accuracy: 0.855
Validation Loss: 4.609649321076217, Validation Accuracy: 0.91
[5/150]: Training Loss: 4.608893507385254, Training Accuracy: 0.965
Validation Loss: 4.610334041012321, Validation Accuracy: 1.15
[6/150]: Training Loss: 4.609225297546387, Training Accuracy: 1.0125
Validation Loss: 4.608022522774472, Validation Accuracy: 1.16
[7/150]: Training Loss: 4.609016616821289, Training Accuracy: 0.9875
Validation Loss: 4.609603547746209, Validation Accuracy: 0.82
[8/150]: Training Loss: 4.609039859008789, Training Accuracy: 1.0025
Validation Loss: 4.60

Test Accuracy,▁▆▅▇▇███▇▇▇▆▆▆▆▆▇▇▇▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
Test Loss,▁▇█▆▅▆▆▇█▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
Train Accuracy,▂▄▄▁▅▇▆▇▆▅▅█▄▂▄▆
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,1.0
Test Loss,51.00808
Train Accuracy,0.9875
Train Loss,4.60864


Hyperparameter with lr:0.01 and wd:0.0004


[1/150]: Training Loss: 4.626314550018311, Training Accuracy: 0.9725
Validation Loss: 4.610879454643103, Validation Accuracy: 0.88
[2/150]: Training Loss: 4.609021481323242, Training Accuracy: 0.9225
Validation Loss: 4.6096283493527945, Validation Accuracy: 1.0
[3/150]: Training Loss: 4.608779692077637, Training Accuracy: 0.9575
Validation Loss: 4.610723838684665, Validation Accuracy: 1.07
[4/150]: Training Loss: 4.609119167327881, Training Accuracy: 0.9125
Validation Loss: 4.60863508236636, Validation Accuracy: 0.89
[5/150]: Training Loss: 4.608814185333252, Training Accuracy: 0.95
Validation Loss: 4.611634175488903, Validation Accuracy: 0.93
[6/150]: Training Loss: 4.6092648048400875, Training Accuracy: 0.8575
Validation Loss: 4.610189516832874, Validation Accuracy: 0.92
[7/150]: Training Loss: 4.6090187705993655, Training Accuracy: 1.005
Validation Loss: 4.610130288798338, Validation Accuracy: 0.88
[8/150]: Training Loss: 4.608886881256104, Training Accuracy: 1.01
Validation Loss: 4

Test Accuracy,▃█▃▁▁▁▃▄▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Test Loss,▁▅▇█▆▆▆▆▇▆▆▆▆▆▆▆▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇
Train Accuracy,▆▄▅▃▅▁▇▇▇▇█▃▄
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,1.08
Test Loss,7.14208
Train Accuracy,0.9225
Train Loss,4.60871


##### Training Using Best Hyperparameters Set

In [23]:
num_epochs = 150
lr = 1e-03
wd = 4e-04

hyperparameters = {'learning_rate': lr,
                    'weight_decay' : wd
                  }

# Load the model
model_1 = LeNet5().to(device)

# Optimizer and scheduler setup
optimizer_1 = torch.optim.AdamW(model_1.parameters(), lr=lr, weight_decay=wd)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_1, T_max=num_epochs)


# Training
run_training(num_epochs, model_1, original_train_loader, original_test_loader, original_test_loader, optimizer_1, scheduler, criterion, device, optimizer_name='AdamW', hyperparameters=hyperparameters, is_wandb = True)

[1/150]: Training Loss: 3.863059723773576, Training Accuracy: 10.28
Validation Loss: 3.371582930255088, Validation Accuracy: 17.92
[2/150]: Training Loss: 3.2885083447941734, Training Accuracy: 19.766
Validation Loss: 3.0681341438536434, Validation Accuracy: 23.43
[3/150]: Training Loss: 3.0068617961595736, Training Accuracy: 25.216
Validation Loss: 2.8509516169311135, Validation Accuracy: 28.09
[4/150]: Training Loss: 2.8243453850221756, Training Accuracy: 28.812
Validation Loss: 2.6608721253218923, Validation Accuracy: 31.85
[5/150]: Training Loss: 2.7198512532826884, Training Accuracy: 30.66
Validation Loss: 2.614684703243766, Validation Accuracy: 32.43
[6/150]: Training Loss: 2.6302451995937415, Training Accuracy: 32.706
Validation Loss: 2.523006957807359, Validation Accuracy: 35.09
[7/150]: Training Loss: 2.5562818654053046, Training Accuracy: 34.01
Validation Loss: 2.4693225895523265, Validation Accuracy: 36.29
[8/150]: Training Loss: 2.503678919409242, Training Accuracy: 35.054


Test Accuracy,█▅▃▂▁▂▁▁▂▁▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁
Test Loss,▁▃▆▃█▆▅▇▅▅▃▃▃▄▄▃▃▃▄▅▆▅▅▄▄▅▅▅▄▄▄▄▄▄▄▄▄▄▃▃
Train Accuracy,▁▂▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
Train Loss,█▆▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,49.47
Test Loss,2.1134
Train Accuracy,65.458
Train Loss,1.1963


# **Large Batch Optimizers**


In [24]:
import torch
from torch.optim.optimizer import Optimizer


class LARS(Optimizer):
    """
    Implements LARS (Layer-wise Adaptive Rate Scaling).

    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
        lr (float): learning rate (default: 1e-3)
        momentum (float, optional): momentum factor (default: 0)
        trust_coef (float, optional): LARS coefficient as used in the paper (default: 1e-3)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
        epsilon (float, optional): epsilon to prevent zero division (default: 0)
    """

    def __init__(
            self,
            params,
            lr: float = 1e-3,
            momentum: float = 0,
            trust_coef: float = 1e-3,
            dampening: float = 0,
            weight_decay: float = 0,
            nesterov=False,
            epsilon: float = 1e-9
    ):
        """
        Initializes a new instance of the LARS optimizer.

        Args:
            params: iterable of parameters to optimize or dicts defining
            lr: learning rate
            momentum: momentum factor
            trust_coef: LARS coefficient as used in the paper
            weight_decay: weight decay (L2 penalty)
            dampening: dampening for momentum
            nesterov: enables Nesterov momentum
            epsilon: epsilon to prevent zero division
        """

        if lr <= 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(
            lr=lr,
            momentum=momentum,
            trust_coef=trust_coef,
            dampening=dampening,
            weight_decay=weight_decay,
            nesterov=nesterov,
            epsilon=epsilon)

        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(LARS, self).__init__(params, defaults)

    def __setstate__(self, state):
        """
        Sets the state of the optimizer.

        Args:
            state: The state to set the optimizer to.
        """
        super(LARS, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def _compute_local_lr(self, p, weight_decay, trust_coef, epsilon):
        """
        Computes the local learning rate for a given parameter.

        Args:
            p: The parameter to compute the local learning rate for.
            weight_decay: The weight decay factor.
            trust_coef: The trust coefficient.
            epsilon: A small constant for numerical stability.

        Returns:
            float: The computed local learning rate.
        """
        w_norm = torch.norm(p.data)
        g_norm = torch.norm(p.grad.data)
        if w_norm * g_norm > 0:
            return trust_coef * w_norm / (g_norm + weight_decay * w_norm + epsilon)
        else:
            return 1

    def _update_params(self, p, d_p, local_lr, lr, momentum, buf,
                       dampening, nesterov, weight_decay):
        """
        Updates the parameters with the computed update.

        Args:
            p: The parameter to be updated.
            d_p: The computed update for the parameter.
            local_lr: The local learning rate.
            lr: The global learning rate.
            momentum: The momentum factor.
            buf: The buffer for the momentum.
            dampening: The dampening for the momentum.
            nesterov: A flag indicating whether to use Nesterov momentum.
            weight_decay: The weight decay factor.
        """
        if weight_decay != 0:
            d_p.add_(weight_decay, p.data)
        if momentum != 0:
            param_state = self.state[p]
            if 'momentum_buffer' not in param_state:
                buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
            else:
                buf = param_state['momentum_buffer']
            buf.mul_(momentum).add_(1 - dampening, d_p)
            if nesterov:
                d_p = d_p.add(momentum, buf)
            else:
                d_p = buf

        p.data.add_(-local_lr * lr, d_p)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            trust_coef = group['trust_coef']
            dampening = group['dampening']
            nesterov = group['nesterov']
            epsilon = group['epsilon']

            for p in group['params']:
                if p.grad is None:
                    continue
                local_lr = self._compute_local_lr(p, weight_decay, trust_coef, epsilon)
                self._update_params(p, p.grad.data, local_lr, group['lr'], momentum, None, dampening, nesterov, weight_decay)

        return loss

In [25]:
import warnings
import torch
from torch.optim import Optimizer
from typing import Optional, Union, Callable, List

class LAMB(Optimizer):
    """
    Implements LAMB (Layer-wise Adaptive Moments) optimizer.

    Args:
        params (iterable): Iterable of parameters to optimize or dicts defining parameter groups.
        learning_rate (Union[float, Callable], optional): The learning rate. Default is 0.001.
        beta_1 (float, optional): The exponential decay rate for the 1st moment estimates. Default is 0.9.
        beta_2 (float, optional): The exponential decay rate for the 2nd moment estimates. Default is 0.999.
        epsilon (float, optional): A small constant for numerical stability. Default is 1e-6.
        weight_decay (float, optional): Weight decay. Default is 0.0.
        exclude_from_weight_decay (Optional[List[str]], optional): List of regex patterns of variables excluded from weight decay. Variables whose name contain a substring matching the pattern will be excluded. Default is None.
        exclude_from_layer_adaptation (Optional[List[str]], optional): List of regex patterns of variables excluded from layer adaptation. Variables whose name contain a substring matching the pattern will be excluded. Default is None.
        name (str, optional): Optional name for the operations created when applying gradients. Defaults to "LAMB".
        **kwargs: Keyword arguments. Allowed to be {`clipnorm`, `clipvalue`, `lr`, `decay`}. `clipnorm` is clip gradients by norm; `clipvalue` is clip gradients by value, `decay` is included for backward compatibility to allow time inverse decay of learning rate. `lr` is included for backward compatibility, recommended to use `learning_rate` instead.

    Note:
        - If "weight_decay_rate" is found in kwargs, it will be renamed to "weight_decay", and will be deprecated in Addons 0.18.
        - If exclude_from_layer_adaptation is None, it will be set to exclude_from_weight_decay.
    """

    def __init__(
        self,
        params,
        lr: Union[float, Callable] = 0.001,
        beta_1: float = 0.9,
        beta_2: float = 0.999,
        epsilon: float = 1e-6,
        weight_decay: float = 0.0,
        exclude_from_weight_decay: Optional[List[str]] = None,
        exclude_from_layer_adaptation: Optional[List[str]] = None,
        name: str = "LAMB",
        **kwargs,
    ):
        """
        Initializes a new instance of the LAMB optimizer.
        """

        if "weight_decay_rate" in kwargs:
            warnings.warn(
                "weight_decay_rate has been renamed to weight_decay,"
                "and will be deprecated in Addons 0.18.",
                DeprecationWarning,
            )
            weight_decay = kwargs["weight_decay_rate"]
            del kwargs["weight_decay_rate"]

        defaults = dict(
            lr=lr,
            betas=(beta_1, beta_2),
            eps=epsilon,
            weight_decay=weight_decay,
            **kwargs)
        super().__init__(params, defaults)

        self.exclude_from_weight_decay = exclude_from_weight_decay
        # exclude_from_layer_adaptation is set to exclude_from_weight_decay if
        # the arg is None.
        if exclude_from_layer_adaptation:
            self.exclude_from_layer_adaptation = exclude_from_layer_adaptation
        else:
            self.exclude_from_layer_adaptation = exclude_from_weight_decay

    def _compute_update(self, p, grad, state, group):
        """
        Computes the update for a given parameter.

        Args:
            p (Tensor): The parameter to be updated.
            grad (Tensor): The gradient of the parameter.
            state (dict): A dictionary containing information about the optimization state.
            group (dict): A dictionary containing the optimization parameters.

        Returns:
            Tensor: The computed update for the parameter.
        """
        # State initialization
        if len(state) == 0:
            state['step'] = 0
            # Exponential moving average of gradient values
            state['exp_avg'] = torch.zeros_like(p.data)
            # Exponential moving average of squared gradient values
            state['exp_avg_sq'] = torch.zeros_like(p.data)

        exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
        beta1, beta2 = group['betas']

        state['step'] += 1

        # Decay the first and second moment running average coefficient
        exp_avg.mul_(beta1).add_(1 - beta1, grad)
        exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

        denom = exp_avg_sq.sqrt().add_(group['eps'])

        update = exp_avg / denom

        # LAMB layer-wise adaptation
        r1 = p.data.pow(2).sum().sqrt()
        r2 = update.pow(2).sum().sqrt()
        r = torch.where(r1 == 0, torch.zeros_like(r1), r1 / r2)

        return r * update

    def _update_params(self, p, update, step_size, weight_decay):
        """
        Updates the parameters with the computed update.

        Args:
            p (Tensor): The parameter to be updated.
            update (Tensor): The computed update for the parameter.
            step_size (float): The step size for the update.
            weight_decay (float): The weight decay factor.
        """
        if weight_decay != 0:
            p.data.add_(-weight_decay * step_size, p.data)

        p.data.add_(-step_size, update)

    def step(self, closure=None):
        """
        Performs a single optimization step.

        Args:
            closure (callable, optional): A closure that reevaluates the model and returns the loss. Default is None.
        """
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('LAMB does not support sparse gradients.')

                state = self.state[p]

                update = self._compute_update(p, grad, state, group)
                self._update_params(p, update, group['lr'], group['weight_decay'])


In [78]:
learning_rates = [1e-02, 5e-02, 1e-01, 5e-01, 1, 1.5, 2]
wd = 1e-03

In [67]:
num_epochs = 150

for lr in learning_rates:

  print('='*50)
  print(f'Hyperparameter with lr:{lr} and wd:{wd}')
  print('='*50)

  hyperparameters = {'learning_rate': lr,
                      'weight_decay' : wd
                      }
  # Load the model
  model = LeNet5().to(device)

  # Optimizer and scheduler setup
  optimizer = LARS(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


  # Training
  run_training(num_epochs,
                model,
                train_loader,
                validation_loader,
                test_loader,
                optimizer,
                scheduler,
                criterion,
                device,
                'LARS-HyperParameterTuning',
                hyperparameters=hyperparameters,
                is_wandb = True,
                n_epochs_stop = 10
  )




Hyperparameter with lr:0.01 and wd:0.001


[1/150]: Training Loss: 4.605613048553467, Training Accuracy: 1.0325
Validation Loss: 4.605481047539195, Validation Accuracy: 1.01
[2/150]: Training Loss: 4.603893094635009, Training Accuracy: 1.2825
Validation Loss: 4.603514659176966, Validation Accuracy: 1.37
[3/150]: Training Loss: 4.601452378845215, Training Accuracy: 1.7625
Validation Loss: 4.600479241389378, Validation Accuracy: 1.96
[4/150]: Training Loss: 4.597398109436035, Training Accuracy: 1.815
Validation Loss: 4.595512341541849, Validation Accuracy: 1.41
[5/150]: Training Loss: 4.590549831390381, Training Accuracy: 1.5175
Validation Loss: 4.587287993947411, Validation Accuracy: 1.68
[6/150]: Training Loss: 4.579249038696289, Training Accuracy: 1.93
Validation Loss: 4.573939116897097, Validation Accuracy: 2.37
[7/150]: Training Loss: 4.560530513763427, Training Accuracy: 2.69
Validation Loss: 4.5522112117451465, Validation Accuracy: 2.64
[8/150]: Training Loss: 4.5309873321533205, Training Accuracy: 2.92
Validation Loss: 4.

Test Accuracy,█▇▂▂▂▂▂▁▁▁▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Test Loss,▁▁▇██▇▇█▇▇▆▇▇▇▇▆▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇
Train Accuracy,▁▁▂▂▃▃▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
Train Loss,████▇▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,5.7
Test Loss,4.83727
Train Accuracy,9.7625
Train Loss,3.99084


Hyperparameter with lr:0.05 and wd:0.001


[1/150]: Training Loss: 4.600790646362305, Training Accuracy: 1.1675
Validation Loss: 4.590667490746565, Validation Accuracy: 1.26
[2/150]: Training Loss: 4.559383809661865, Training Accuracy: 2.1575
Validation Loss: 4.506907958133965, Validation Accuracy: 2.54
[3/150]: Training Loss: 4.4036731437683105, Training Accuracy: 3.8525
Validation Loss: 4.321598778864381, Validation Accuracy: 4.84
[4/150]: Training Loss: 4.246265041351318, Training Accuracy: 5.2175
Validation Loss: 4.227905522486207, Validation Accuracy: 6.02
[5/150]: Training Loss: 4.176178216934204, Training Accuracy: 6.1675
Validation Loss: 4.179626170237353, Validation Accuracy: 6.38
[6/150]: Training Loss: 4.135982835769654, Training Accuracy: 6.6675
Validation Loss: 4.148335417364813, Validation Accuracy: 6.64
[7/150]: Training Loss: 4.108450821304321, Training Accuracy: 7.2275
Validation Loss: 4.130975275282648, Validation Accuracy: 6.89
[8/150]: Training Loss: 4.0844139430999755, Training Accuracy: 7.6
Validation Loss

...

[1/150]: Training Loss: 4.426865048217773, Training Accuracy: 2.45
Validation Loss: 4.219578746018136, Validation Accuracy: 4.19
[2/150]: Training Loss: 4.060279036712647, Training Accuracy: 6.7225
Validation Loss: 3.8940811293899635, Validation Accuracy: 9.43
[3/150]: Training Loss: 3.7707011901855467, Training Accuracy: 11.6925
Validation Loss: 3.8030508642743346, Validation Accuracy: 11.0
[4/150]: Training Loss: 3.58028080368042, Training Accuracy: 15.05
Validation Loss: 3.516739834645751, Validation Accuracy: 16.1
[5/150]: Training Loss: 3.4152388591766356, Training Accuracy: 17.82
Validation Loss: 3.411277610025588, Validation Accuracy: 17.62
[6/150]: Training Loss: 3.2448327434539794, Training Accuracy: 20.7725
Validation Loss: 3.2892891753251385, Validation Accuracy: 19.86
[7/150]: Training Loss: 3.1129616333007815, Training Accuracy: 23.41
Validation Loss: 3.155244686041668, Validation Accuracy: 22.75
[8/150]: Training Loss: 2.9936462223052978, Training Accuracy: 25.73
Validati

Test Accuracy,█▆▁▄▃▄▄▄▄▄▄▄▄▄▄▅▅▄▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅
Test Loss,▁▅▇▇▇▆▇██▇▇█████████████████████████████
Train Accuracy,▁▁▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Train Loss,█▇▇▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
Test Accuracy,15.72
Test Loss,20.29688
Train Accuracy,69.42
Train Loss,1.01537


Hyperparameter with lr:2 and wd:0.001


[1/150]: Training Loss: 4.6014143745422365, Training Accuracy: 0.98
Validation Loss: 4.606205284215842, Validation Accuracy: 0.91
[2/150]: Training Loss: 4.605726162719726, Training Accuracy: 0.93
Validation Loss: 4.606422876856129, Validation Accuracy: 0.91
[3/150]: Training Loss: 4.605443458557129, Training Accuracy: 0.9375
Validation Loss: 4.606579792727331, Validation Accuracy: 0.82
[4/150]: Training Loss: 4.6053140991210935, Training Accuracy: 1.015
Validation Loss: 4.606754381945179, Validation Accuracy: 0.82
[5/150]: Training Loss: 4.6052259376525875, Training Accuracy: 1.0275
Validation Loss: 4.60688726765335, Validation Accuracy: 0.82
[6/150]: Training Loss: 4.605191477966309, Training Accuracy: 1.0175
Validation Loss: 4.607019655264107, Validation Accuracy: 0.82
[7/150]: Training Loss: 4.6051658882141115, Training Accuracy: 1.015
Validation Loss: 4.607097844409335, Validation Accuracy: 0.82
[8/150]: Training Loss: 4.605149390411377, Training Accuracy: 0.9775
Validation Loss: 

Test Accuracy,▁▆███▆▇▇▇▇▇▇▇▇▆▇███▇████████████████████
Test Loss,█▅▁▁▃▃▅▄▄▄▄▄▄▄▄▅▅▅▅▅▆▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅
Train Accuracy,▅▁▂▇█▇▇▄▂▇▃
Train Loss,▁██▇▇▇▇▇▇▇▇
Test Accuracy,1.0
Test Loss,4.60539
Train Accuracy,0.9625
Train Loss,4.60514


In [26]:
num_epochs = 150
lr = 1.5
wd = 1e-03

hyperparameters = {'learning_rate': lr,
                    'weight_decay' : wd
                  }

# Load the model
model = LeNet5().to(device)

# Optimizer and scheduler setup
optimizer = LARS(model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


# Training
run_training(
    num_epochs,
    model,
    original_train_loader,
    original_test_loader,
    original_test_loader,
    optimizer,
    scheduler,
    criterion,
    device,
    optimizer_name='LARS',
    hyperparameters=hyperparameters,
    is_wandb = True
)

/tmp/ipykernel_1584951/4263216451.py:113: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1578.)
  d_p.add_(weight_decay, p.data)


[1/150]: Training Loss: 4.004822687724667, Training Accuracy: 8.156
Validation Loss: 3.6169370329304105, Validation Accuracy: 15.1
[2/150]: Training Loss: 3.489779775099986, Training Accuracy: 16.288
Validation Loss: 3.195649910884298, Validation Accuracy: 21.71
[3/150]: Training Loss: 3.188335987003258, Training Accuracy: 21.346
Validation Loss: 2.9737777011409685, Validation Accuracy: 26.15
[4/150]: Training Loss: 3.0118914528576006, Training Accuracy: 24.982
Validation Loss: 2.7995891707717995, Validation Accuracy: 29.11
[5/150]: Training Loss: 2.8688232484071152, Training Accuracy: 28.08
Validation Loss: 2.7174989600090464, Validation Accuracy: 30.23
[6/150]: Training Loss: 2.7214920926276984, Training Accuracy: 30.662
Validation Loss: 2.5305145515757763, Validation Accuracy: 34.82
[7/150]: Training Loss: 2.6200312084858983, Training Accuracy: 32.576
Validation Loss: 2.474041129373441, Validation Accuracy: 35.97
[8/150]: Training Loss: 2.53857664821093, Training Accuracy: 34.358
Va

Test Accuracy,▆█▃▄▃▂▂▂▂▂▂▂▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,▁▃▆▅██▇█▆▇▆▅▆▇█▇▆▇▆▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▆▆▇▆▆▆
Train Accuracy,▁▂▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████████
Train Loss,█▇▆▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Test Accuracy,56.63
Test Loss,1.94981
Train Accuracy,85.464
Train Loss,0.49865


In [28]:
# check the lr updates from paper 18

num_epochs = 150
lr = 1.5
wd = 1e-03
batch_sizes = [512, 1024, 2048, 4096, 8192, 16384 , 32768]

for batch_size in batch_sizes:
  # Root square scale-up of learning rate
  k = (batch_size / 64.0) ** 0.5
  print('='*50)
  print(f'Batch size: {batch_size}, Learning rate: {lr*k}, Weight decay: {wd}')
  print('='*50)

  hyperparameters = {
    'batch_size': batch_size,
    'learning_rate': lr*k,
    'weight_decay' : wd
  }
  if batch_size <= 4096:
  
    # load data
    data = CIFAR100Data(batch_size= batch_size)
    original_train_loader_large_batch, original_test_loader_large_batch = data.train_test()
    # Load the model
    model = LeNet5().to(device)
    # Optimizer and scheduler setup
    optimizer = LARS(model.parameters(), lr= lr*k, weight_decay=wd, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


    # Training
    run_training(
        num_epochs,
        model,
        original_train_loader_large_batch,
        original_test_loader_large_batch,
        original_test_loader_large_batch,
        optimizer,
        scheduler,
        criterion,
        device,
        optimizer_name='LARS_Large_Batches',
        hyperparameters=hyperparameters,
        is_wandb = True
    )
  else:
    # load data
    data = CIFAR100Data(batch_size= 4096)
    original_train_loader_large_batch, original_test_loader_large_batch = data.train_test()
    # Load the model
    model = LeNet5().to(device)
    # Optimizer and scheduler setup
    optimizer = LARS(model.parameters(), lr= lr*k, weight_decay=wd, momentum=0.9)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    accumulation_steps = batch_size // 4096

    # Training
    run_training(
        num_epochs,
        model,
        original_train_loader_large_batch,
        original_test_loader_large_batch,
        original_test_loader_large_batch,
        optimizer,
        scheduler,
        criterion,
        device,
        optimizer_name='LARS_Large_Batches',
        accumulation_steps=accumulation_steps,
        hyperparameters=hyperparameters,
        is_wandb = True
    )

Batch size: 512, Learning rate: 4.242640687119286, Weight decay: 0.001


[1/150]: Training Loss: 4.194531238808924, Training Accuracy: 5.802
Validation Loss: 3.8219263911247254, Validation Accuracy: 11.06
[2/150]: Training Loss: 3.737521487839368, Training Accuracy: 12.162
Validation Loss: 3.5326468467712404, Validation Accuracy: 15.5
[3/150]: Training Loss: 3.469568571265863, Training Accuracy: 16.654
Validation Loss: 3.2325081706047056, Validation Accuracy: 21.14
[4/150]: Training Loss: 3.2623822543085836, Training Accuracy: 20.338
Validation Loss: 3.041664385795593, Validation Accuracy: 24.34
[5/150]: Training Loss: 3.1202199824002324, Training Accuracy: 23.134
Validation Loss: 2.9670259952545166, Validation Accuracy: 26.23
[6/150]: Training Loss: 2.973516508024566, Training Accuracy: 25.864
Validation Loss: 2.835892844200134, Validation Accuracy: 28.97
[7/150]: Training Loss: 2.8618772808386357, Training Accuracy: 27.73
Validation Loss: 2.731530475616455, Validation Accuracy: 31.08
[8/150]: Training Loss: 2.7622045083921782, Training Accuracy: 29.856
Va

Test Accuracy,█▆▁▅▄▅▃▆▄▃▃▅▅▅▅▅▆▆▆▆
Test Loss,▆▆█▄▃▃▄▂▂▃▂▃▃▂▃▃▁▂▁▁
Train Accuracy,▁▂▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇█████████
Train Loss,█▇▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,56.18
Test Loss,1.97009
Train Accuracy,84.57
Train Loss,0.5342


Batch size: 1024, Learning rate: 6.0, Weight decay: 0.001


[1/150]: Training Loss: 4.271560571631607, Training Accuracy: 4.982
Validation Loss: 3.9391962051391602, Validation Accuracy: 8.84
[2/150]: Training Loss: 3.82728639914065, Training Accuracy: 10.948
Validation Loss: 3.6827336311340333, Validation Accuracy: 13.37
[3/150]: Training Loss: 3.6269987845907408, Training Accuracy: 14.39
Validation Loss: 3.4175909042358397, Validation Accuracy: 18.27
[4/150]: Training Loss: 3.4358463238696664, Training Accuracy: 17.696
Validation Loss: 3.2815504550933836, Validation Accuracy: 20.38
[5/150]: Training Loss: 3.3039202203555984, Training Accuracy: 19.798
Validation Loss: 3.171510195732117, Validation Accuracy: 23.09
[6/150]: Training Loss: 3.1788000671231016, Training Accuracy: 22.106
Validation Loss: 3.083156633377075, Validation Accuracy: 24.25
[7/150]: Training Loss: 3.0781905553778826, Training Accuracy: 24.04
Validation Loss: 2.9470993995666506, Validation Accuracy: 26.99
[8/150]: Training Loss: 2.9905799846259917, Training Accuracy: 25.462
V

...

[1/150]: Training Loss: 4.5802256144010105, Training Accuracy: 1.524
Validation Loss: 4.505903720855713, Validation Accuracy: 3.24
[2/150]: Training Loss: 4.487952305720403, Training Accuracy: 2.782
Validation Loss: 4.534458001454671, Validation Accuracy: 1.82
[3/150]: Training Loss: 4.498784615443303, Training Accuracy: 2.232
Validation Loss: 4.477282365163167, Validation Accuracy: 2.08
[4/150]: Training Loss: 4.4941191306481, Training Accuracy: 2.178
Validation Loss: 4.612959861755371, Validation Accuracy: 1.29
[5/150]: Training Loss: 4.5912819642287035, Training Accuracy: 1.504
Validation Loss: 5.598563989003499, Validation Accuracy: 2.19
[6/150]: Training Loss: 5.028410141284649, Training Accuracy: 1.632
Validation Loss: 7.806542873382568, Validation Accuracy: 1.07
[7/150]: Training Loss: 14089.29747926272, Training Accuracy: 1.044
Validation Loss: 7.282219409942627, Validation Accuracy: 0.93
[8/150]: Training Loss: 461533512.8043683, Training Accuracy: 0.978
Validation Loss: 14518

Test Accuracy,█▃▁
Train Accuracy,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,▁▁▁████
Test Accuracy,1.0
Test Loss,nan
Train Accuracy,1.0
Train Loss,nan


Batch size: 32768, Learning rate: 33.941125496954285, Weight decay: 0.001


[1/150]: Training Loss: 4.600700745215783, Training Accuracy: 1.18
Validation Loss: 4.5345486005147295, Validation Accuracy: 2.25
[2/150]: Training Loss: 4.525542259216309, Training Accuracy: 2.194
Validation Loss: 4.480770270029704, Validation Accuracy: 3.63
[3/150]: Training Loss: 4.467245468726525, Training Accuracy: 3.348
Validation Loss: 4.682004292805989, Validation Accuracy: 2.02
[4/150]: Training Loss: 4.584587500645564, Training Accuracy: 2.316
Validation Loss: 4.589324951171875, Validation Accuracy: 1.07
[5/150]: Training Loss: 4.593533919407771, Training Accuracy: 1.128
Validation Loss: 4.565906047821045, Validation Accuracy: 2.24
[6/150]: Training Loss: 4.610266098609338, Training Accuracy: 1.65
Validation Loss: 5.121236960093181, Validation Accuracy: 1.01
[7/150]: Training Loss: 5.261244590465839, Training Accuracy: 1.226
Validation Loss: 7.05544392267863, Validation Accuracy: 1.19
[8/150]: Training Loss: 6.352289456587571, Training Accuracy: 1.176
Validation Loss: 22.8200

Test Accuracy,█▃▁
Train Accuracy,▇█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,▁▁▁▁█
Test Accuracy,1.0
Test Loss,nan
Train Accuracy,1.0
Train Loss,nan


In [30]:
num_epochs = 150
learning_rates = [9e-04, 95e-05, 1e-03, 15e-04, 2e-03]
wd = 4e-04
for lr in learning_rates:

  print('='*50)
  print(f'Hyperparameter with lr:{lr} and wd:{wd}')
  print('='*50)

  hyperparameters = {'learning_rate': lr,
                      'weight_decay' : wd
                      }
  # Load the model
  model = LeNet5().to(device)

  # Optimizer and scheduler setup
  optimizer = LAMB(model.parameters(), lr=lr, weight_decay=wd)
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


  # Training
  run_training(num_epochs,
                model,
                train_loader,
                validation_loader,
                test_loader,
                optimizer,
                scheduler,
                criterion,
                device,
                'LAMB-HyperParameterTuning',
                hyperparameters=hyperparameters,
                is_wandb = True,
                n_epochs_stop = 10
  )




Hyperparameter with lr:0.0009 and wd:0.0004


[1/150]: Training Loss: 4.366185345458985, Training Accuracy: 3.775
Validation Loss: 4.140061999582182, Validation Accuracy: 6.19
[2/150]: Training Loss: 3.9872212955474855, Training Accuracy: 9.2825
Validation Loss: 3.9175391182018693, Validation Accuracy: 10.13
[3/150]: Training Loss: 3.7751855819702147, Training Accuracy: 12.73
Validation Loss: 3.754828929901123, Validation Accuracy: 13.44
[4/150]: Training Loss: 3.6031822479248046, Training Accuracy: 15.7675
Validation Loss: 3.5609892857302525, Validation Accuracy: 16.33
[5/150]: Training Loss: 3.4478089206695555, Training Accuracy: 18.66
Validation Loss: 3.4119288389849816, Validation Accuracy: 19.45
[6/150]: Training Loss: 3.3203197284698485, Training Accuracy: 20.7075
Validation Loss: 3.349654642639646, Validation Accuracy: 20.47
[7/150]: Training Loss: 3.20732957611084, Training Accuracy: 22.7475
Validation Loss: 3.2386614349996967, Validation Accuracy: 22.79
[8/150]: Training Loss: 3.115479539489746, Training Accuracy: 24.5875

Test Accuracy,▃█▁▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▂▃▃▃▃▃▃▃▂▃▃
Test Loss,▆▁▂▂▄▄▆█▇▇▆▇█▇▇▇▆▇▇▇▇▇▆▆▆▆▆▆▆▇▇▆▆▆▆▆▆▆▆▆
Train Accuracy,▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
Train Loss,█▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
Test Accuracy,14.49
Test Loss,17.67243
Train Accuracy,70.73
Train Loss,1.0265


Hyperparameter with lr:0.00095 and wd:0.0004


[1/150]: Training Loss: 4.36243955039978, Training Accuracy: 3.8475
Validation Loss: 4.152171432592307, Validation Accuracy: 6.23
[2/150]: Training Loss: 3.988539717102051, Training Accuracy: 9.035
Validation Loss: 3.8899211443153914, Validation Accuracy: 10.42
[3/150]: Training Loss: 3.784552033996582, Training Accuracy: 12.44
Validation Loss: 3.716481655266634, Validation Accuracy: 13.63
[4/150]: Training Loss: 3.6115343948364256, Training Accuracy: 15.38
Validation Loss: 3.575136796684022, Validation Accuracy: 15.25
[5/150]: Training Loss: 3.464410584640503, Training Accuracy: 18.1625
Validation Loss: 3.448981456695848, Validation Accuracy: 17.92
[6/150]: Training Loss: 3.340861389923096, Training Accuracy: 20.2875
Validation Loss: 3.3443257702384024, Validation Accuracy: 19.79
[7/150]: Training Loss: 3.240240854263306, Training Accuracy: 22.0975
Validation Loss: 3.2861741910314866, Validation Accuracy: 21.07
[8/150]: Training Loss: 3.1493673683166503, Training Accuracy: 23.8725
Val

...

[1/150]: Training Loss: 4.262849729156494, Training Accuracy: 4.9725
Validation Loss: 3.9988777105975304, Validation Accuracy: 9.32
[2/150]: Training Loss: 3.787733755874634, Training Accuracy: 12.5275
Validation Loss: 3.666901140455987, Validation Accuracy: 14.13
[3/150]: Training Loss: 3.5222632232666014, Training Accuracy: 16.975
Validation Loss: 3.430799526773441, Validation Accuracy: 18.28
[4/150]: Training Loss: 3.3261954177856445, Training Accuracy: 20.0275
Validation Loss: 3.268884590476941, Validation Accuracy: 20.25
[5/150]: Training Loss: 3.1651099575042725, Training Accuracy: 22.825
Validation Loss: 3.140599861266507, Validation Accuracy: 23.63
[6/150]: Training Loss: 3.0220409996032713, Training Accuracy: 25.57
Validation Loss: 3.062152488975768, Validation Accuracy: 24.92
[7/150]: Training Loss: 2.9046311878204345, Training Accuracy: 27.855
Validation Loss: 2.9628464735237654, Validation Accuracy: 26.95
[8/150]: Training Loss: 2.7910710647583006, Training Accuracy: 30.235

Test Accuracy,▅█▂▅▄▂▁▁▁▁▂▃▃▂▂▃▃▃▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▃▄▄▄▄▄▄
Test Loss,▁▄▇▆▇▇███▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆
Train Accuracy,▁▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
Train Loss,█▇▇▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
Test Accuracy,15.7
Test Loss,32.80274
Train Accuracy,82.41
Train Loss,0.55922


Hyperparameter with lr:0.002 and wd:0.0004


[1/150]: Training Loss: 4.222224911499024, Training Accuracy: 5.3975
Validation Loss: 3.937301506662065, Validation Accuracy: 9.22
[2/150]: Training Loss: 3.7261308227539063, Training Accuracy: 13.09
Validation Loss: 3.593827167134376, Validation Accuracy: 15.34
[3/150]: Training Loss: 3.4419474758148194, Training Accuracy: 18.2775
Validation Loss: 3.3379262875599465, Validation Accuracy: 19.89
[4/150]: Training Loss: 3.242010182952881, Training Accuracy: 21.865
Validation Loss: 3.2397833553848754, Validation Accuracy: 21.54
[5/150]: Training Loss: 3.074250465774536, Training Accuracy: 24.8525
Validation Loss: 3.2218855049959414, Validation Accuracy: 22.29
[6/150]: Training Loss: 2.9360343181610107, Training Accuracy: 27.3175
Validation Loss: 2.9865669110778033, Validation Accuracy: 26.78
[7/150]: Training Loss: 2.801424619293213, Training Accuracy: 30.0175
Validation Loss: 2.881270451150882, Validation Accuracy: 28.82
[8/150]: Training Loss: 2.688298546409607, Training Accuracy: 32.05

Test Accuracy,█▅▁▂▁▁▁▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▂▂▂▂▂▂▂▂▂
Test Loss,▁▃▆▅▆▇▇▇▇▇▇▇▇███████████████████████████
Train Accuracy,▁▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇███
Train Loss,█▇▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
Test Accuracy,11.85
Test Loss,35.52837
Train Accuracy,78.7575
Train Loss,0.68238


In [32]:
num_epochs = 150
lr = 4.8/(2**5 *1e02) # approximately 15e-04
wd = 4e-04

hyperparameters = {'learning_rate': lr,
                    'weight_decay' : wd
                  }

# Load the model
model = LeNet5().to(device)

# Optimizer and scheduler setup
optimizer = LAMB(model.parameters(), learning_rate=lr, weight_decay=wd)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


# Training
run_training(num_epochs, model, original_train_loader, original_test_loader, original_test_loader, optimizer, scheduler, criterion, device, optimizer_name='LAMB', hyperparameters=hyperparameters, is_wandb = True)

Train Accuracy,▁▄▆▇█
Train Loss,█▅▃▂▁
Train Accuracy,22.668
Train Loss,3.18342


[1/150]: Training Loss: 4.170614353226274, Training Accuracy: 6.554
Validation Loss: 3.8419871087286883, Validation Accuracy: 11.55
[2/150]: Training Loss: 3.73113738697813, Training Accuracy: 13.326
Validation Loss: 3.5141605601948536, Validation Accuracy: 17.24
[3/150]: Training Loss: 3.4963422099037853, Training Accuracy: 17.148
Validation Loss: 3.329459586720558, Validation Accuracy: 21.14
[4/150]: Training Loss: 3.32537763838268, Training Accuracy: 20.17
Validation Loss: 3.1594036445496188, Validation Accuracy: 23.38
[5/150]: Training Loss: 3.1821046982274948, Training Accuracy: 22.724
Validation Loss: 3.024734082495331, Validation Accuracy: 25.42
[6/150]: Training Loss: 3.0576654736648132, Training Accuracy: 24.838
Validation Loss: 2.917888655024729, Validation Accuracy: 27.48
[7/150]: Training Loss: 2.9461568444586166, Training Accuracy: 26.996
Validation Loss: 2.8047209317517128, Validation Accuracy: 29.77
[8/150]: Training Loss: 2.860208951298843, Training Accuracy: 28.704
Val

Test Accuracy,▆█▁▁▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂
Test Loss,▂▁█▆▇▅▆▇▄▅▅▅▅▅▆▅▅▅▅▆▆▆▅▅▅▅▆▆▅▅▅▅▅▅▄▅▅▅▄▄
Train Accuracy,▁▂▃▃▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████████████
Train Loss,█▇▆▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,53.01
Test Loss,2.21681
Train Accuracy,78.148
Train Loss,0.74618


In [33]:
# check the lr updates from paper 18

num_epochs = 150
wd = 1e-04
batch_sizes = [512, 1024, 2048, 4096, 8192, 16384 , 32768]
learning_rates = [4.8/(2**i * 10**3) for i in reversed([x * 0.5 for x in range(0, 8)])]

for worker, batch_size in enumerate(batch_sizes):
  print('='*50)
  print(f'Batch size: {batch_size}')
  print('='*50)

  hyperparameters = {
    'batch_size': batch_size,
    'learning_rate': learning_rates[worker],
    'weight_decay' : wd
  }

  if batch_size <= 4096:
    # load data
    data = CIFAR100Data(batch_size= batch_size)
    original_train_loader_large_batch, original_test_loader_large_batch = data.train_test()
    # Load the model
    model = LeNet5().to(device)
    # Optimizer and scheduler setup
    optimizer = LAMB(model.parameters(), learning_rate=learning_rates[worker], weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


    # Training
    run_training(
        num_epochs,
        model,
        original_train_loader_large_batch,
        original_test_loader_large_batch,
        original_test_loader_large_batch,
        optimizer,
        scheduler,
        criterion,
        device,
        optimizer_name='LAMB_Large_Batches',
        hyperparameters=hyperparameters,
        is_wandb = True
    )
  else:
    accumulation_steps = batch_size // 4096
    # load data
    data = CIFAR100Data(batch_size= batch_size)
    original_train_loader_large_batch, original_test_loader_large_batch = data.train_test()
    # Load the model
    model = LeNet5().to(device)
    # Optimizer and scheduler setup
    optimizer = LAMB(model.parameters(), learning_rate=learning_rates[worker], weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)


    # Training
    run_training(
        num_epochs,
        model,
        original_train_loader_large_batch,
        original_test_loader_large_batch,
        original_test_loader_large_batch,
        optimizer,
        scheduler,
        criterion,
        device,
        optimizer_name='LAMB_Large_Batches',
        accumulation_steps=accumulation_steps,
        hyperparameters=hyperparameters,
        is_wandb = True
    )

Batch size: 512


[1/150]: Training Loss: 4.548603617415136, Training Accuracy: 2.498
Validation Loss: 4.416820383071899, Validation Accuracy: 4.51
[2/150]: Training Loss: 4.243599555930313, Training Accuracy: 5.9
Validation Loss: 4.063300430774689, Validation Accuracy: 8.11
[3/150]: Training Loss: 4.025548063978857, Training Accuracy: 8.684
Validation Loss: 3.9094903230667115, Validation Accuracy: 10.6
[4/150]: Training Loss: 3.9005869870283165, Training Accuracy: 10.854
Validation Loss: 3.7815954089164734, Validation Accuracy: 13.47
[5/150]: Training Loss: 3.7978148922628288, Training Accuracy: 12.58
Validation Loss: 3.688620662689209, Validation Accuracy: 14.4
[6/150]: Training Loss: 3.708504837386462, Training Accuracy: 14.016
Validation Loss: 3.5979228973388673, Validation Accuracy: 15.97
[7/150]: Training Loss: 3.6287196242079442, Training Accuracy: 15.174
Validation Loss: 3.5210944056510924, Validation Accuracy: 17.53
[8/150]: Training Loss: 3.5536105267855587, Training Accuracy: 16.508
Validatio

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x74777b725b40>
Traceback (most recent call last):
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
Exception ignored in:     Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x74777b725b40>if w.is_alive():
<function _MultiProcessingDataLoaderIter.__del__ at 0x74777b725b40>Traceback (most recent call last):
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__


  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
        self._shutdown_workers()Traceback (most recent call last):
assert self._parent_pid == os.getpid(), 

[13/150]: Training Loss: 3.2494236060551236, Training Accuracy: 21.83
Validation Loss: 3.150032651424408, Validation Accuracy: 24.52


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x74777b725b40>
Traceback (most recent call last):
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1462, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
Exception ignored in:     assert self._parent_pid == os.getpid(), 'can only test a child process'<function _MultiProcessingDataLoaderIter.__del__ at 0x74777b725b40>
AssertionError: can only test a child process

Traceback (most recent call last):
  File "/home/ali/Repos/University/MLDL/Project/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "/home/ali/Repos/University/MLDL/Projec

...

[1/150]: Training Loss: 4.603197281177227, Training Accuracy: 1.246
Validation Loss: 4.599034945170085, Validation Accuracy: 2.16
[2/150]: Training Loss: 4.594296822181115, Training Accuracy: 2.678
Validation Loss: 4.58727502822876, Validation Accuracy: 3.46
[3/150]: Training Loss: 4.580152915074275, Training Accuracy: 3.29
Validation Loss: 4.5680915514628095, Validation Accuracy: 3.75
[4/150]: Training Loss: 4.55770389850323, Training Accuracy: 3.57
Validation Loss: 4.539859771728516, Validation Accuracy: 4.09
[5/150]: Training Loss: 4.526640048393836, Training Accuracy: 3.846
Validation Loss: 4.5020168622334795, Validation Accuracy: 4.4
[6/150]: Training Loss: 4.4863302157475395, Training Accuracy: 4.116
Validation Loss: 4.454424858093262, Validation Accuracy: 4.5
[7/150]: Training Loss: 4.436847613408015, Training Accuracy: 4.154
Validation Loss: 4.398792107899983, Validation Accuracy: 4.51
[8/150]: Training Loss: 4.382756233215332, Training Accuracy: 4.368
Validation Loss: 4.336817

Test Accuracy,█▁▃
Test Loss,▁█▂
Train Accuracy,▁▁▁▂▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██████████████
Train Loss,██▇▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,28.68
Test Loss,2.9212
Train Accuracy,27.018
Train Loss,3.01049


Batch size: 8192


[1/150]: Training Loss: 4.605092797960554, Training Accuracy: 1.07
Validation Loss: 4.604321002960205, Validation Accuracy: 1.5


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.53 GiB. GPU 

# **Distributed Approaches**

In [12]:
# Initialize a model and save its initial parameters
initial_model = LeNet5()
initial_state_dict = initial_model.state_dict()

### Local SGD

In [29]:
from torch.optim import Optimizer

import torch
from torch.optim import Optimizer
import torch
from torch.optim import Optimizer

class LocalSGDOptimizer(Optimizer):
    def __init__(self, global_model, lr=0.01):
        self.global_model = global_model
        self.lr = lr
        params = list(global_model.parameters())
        super(LocalSGDOptimizer, self).__init__(params, {'lr': lr})

    def step(self, local_models):
        # Get global model parameters
        global_params = list(self.global_model.parameters())
        
        # Initialize deltas for each parameter, same size as global parameters
        deltas = [torch.zeros_like(param) for param in global_params]
        
        # Sum up differences between global model and local models
        for local_model in local_models:
            local_params = list(local_model.parameters())
            for i, param in enumerate(local_params):
                deltas[i] += (global_params[i] - param)
        
        # Average the delta over the number of local models
        num_models = len(local_models)
        for i, delta in enumerate(deltas):
            deltas[i] /= self.lr
            deltas[i] /= num_models
        
        # Update global model parameters
        with torch.no_grad():
            for i, param in enumerate(global_params):
                param.copy_(param - self.lr * deltas[i])
        
        return None






In [27]:
import time
from datetime import timedelta

def local_SGD(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs):
  total_start_time = time.time()

  iterations = num_epochs // j
  # Initialize a model with same value of param for each chunk
  local_models = [LeNet5().to(device) for _ in range(k)]
  for model in local_models:
    model.load_state_dict(initial_state_dict)
  #Initialize optimizers for each chunk
  local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd) for model in local_models]
  # Initialize the global model
  global_model = LeNet5().to(device)
  global_model.load_state_dict(initial_state_dict)
  
  global_optimizer = LocalSGDOptimizer(global_model, lr)
  
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(global_optimizer, T_max=iterations)


  for iteration in range(iterations):
    for worker, shard_loader in enumerate(shard_loaders):
      train_start_time = time.time()
      for loca_step in range(j):
        train_loss, train_accuracy = train(local_models[worker], shard_loader, local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
        print(f'Worker {worker+1}, [{loca_step+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      train_end_time = time.time()
      print(f'Time taken for training worker {worker+1}: {str(timedelta(seconds=train_end_time - train_start_time))}')
      print('-'*50)
    sync_start_time = time.time()

    global_optimizer.step(local_models)

    scheduler.step()

    for local_optimizer in local_optimizers:
        # print(f"{local_optimizer.param_groups[0]['lr']}-->{global_optimizer.param_groups[0]['lr']}") 
        local_optimizer.param_groups[0]['lr'] = global_optimizer.param_groups[0]['lr']


    for local_model in local_models:
      local_model.load_state_dict(global_model.state_dict())
    sync_end_time = time.time()
    print('*'*50)
    print(f'Time taken for synchronization: {str(timedelta(seconds=sync_end_time - sync_start_time))}')
    test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
    print(f'Local Step {iteration+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
    print('*'*50)
    
# TODO: Schedule ro check konim 

  total_end_time = time.time()
  print('/'*50)
  print(f'Total time taken for local_SGD: {str(timedelta(seconds=total_end_time - total_start_time))}')
  print('/'*50)

In [30]:
lr = 1e-02
wd = 1e-03
K = [2, 4, 8]
J = [4, 8, 16, 32, 64]
num_epochs = 150
data = CIFAR100Data()
loss_fn = nn.CrossEntropyLoss()

for k in K: # Number of workers
  shard_loaders = data.iid_shards(num_shards=k)
  for j in J:
    print('='*50)
    print(f'Number of Workers:{k}, Number of Local Steps:{j}')
    print('='*50)
    local_SGD(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Number of Workers:2, Number of Local Steps:1
Worker 1, [01/01]: Training Loss: 4.349158551, Training Accuracy: 3.852
Time taken for training worker 1: 0:00:14.840832
--------------------------------------------------
Worker 2, [01/01]: Training Loss: 4.341307805, Training Accuracy: 4.244
Time taken for training worker 2: 0:00:12.196546
--------------------------------------------------
**************************************************
Time taken for synchronization: 0:00:00.002010
Local Step 01: Test Loss: 3.975554096, Test Accuracy: 9.000
**************************************************
Worker 1, [01/01]: Training Loss: 3.868072952, Training Accuracy: 9.948
Time taken for training worker 1: 0:00:12.951741
--------------------------------------------------
Worker 2, [01/01]: Training Loss: 3.855989874, Training Accuracy: 10.432
Time taken for training worker 2: 0:00:13.916281
--------------------------------

KeyboardInterrupt: 

### SlowMo Implementation

In [ ]:
from torch.optim import Optimizer
class SlowMoOptimizer(Optimizer):
    def __init__(self, global_model, momentum, lr=0.01, beta=0.5, alpha=1.0):
        self.global_model = global_model
        self.lr = lr
        self.beta = beta
        self.alpha = alpha
        self.momentum = momentum
        params = list(global_model.parameters())
        super(SlowMoOptimizer, self).__init__(params, {'lr': lr})
        
    def step(self, local_models):
        # Calculate exact average of local models
        avg_state_dict = {key: torch.zeros_like(value) for key, value in local_models[0].state_dict().items()}
        for model in local_models:
            for key, param in model.state_dict().items():
                avg_state_dict[key] += param
        
        # Averaging the models
        for key in avg_state_dict:
            avg_state_dict[key] /= len(local_models)
        
        # Perform SlowMo momentum update
        for key in self.global_model.state_dict().keys():
            self.momentum[key] = self.beta * self.momentum[key] + (1.0 / self.lr) * (self.global_model.state_dict()[key] - avg_state_dict[key])
        
        # Update global model parameters with outer update
        with torch.no_grad():
            for key, param in self.global_model.state_dict().items():
                param.copy_(param - self.alpha * self.lr * self.momentum[key])

        return None


In [30]:
import time
from datetime import timedelta
import time
from datetime import timedelta

def local_SGD_SlowMo(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs, beta, alpha):
  total_start_time = time.time()

  iterations = num_epochs // j
  # Initialize a model with same value of param for each chunk
  local_models = [LeNet5().to(device) for _ in range(k)]
  for model in local_models:
    model.load_state_dict(initial_state_dict)
  #Initialize optimizers for each chunk
  local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd) for model in local_models]
  # Initialize the global model
  global_model = LeNet5().to(device)
  global_model.load_state_dict(initial_state_dict)
  
  momentum = {key: torch.zeros_like(value) for key, value in global_model.state_dict().items()}
  
  global_optimizer = SlowMoOptimizer(global_model, momentum , lr, beta, alpha)
  
  scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(global_optimizer, T_max=iterations)


  for iteration in range(iterations):
    for worker, shard_loader in enumerate(shard_loaders):
      train_start_time = time.time()
      for loca_step in range(j):
        train_loss, train_accuracy = train(local_models[worker], shard_loader, local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
        print(f'Worker {worker+1}, [{loca_step+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      train_end_time = time.time()
      print(f'Time taken for training worker {worker+1}: {str(timedelta(seconds=train_end_time - train_start_time))}')
      print('-'*50)
    sync_start_time = time.time()

    global_optimizer.step(local_models)

    scheduler.step()

    for local_optimizer in local_optimizers:
        # print(f"{local_optimizer.param_groups[0]['lr']}-->{global_optimizer.param_groups[0]['lr']}") 
        local_optimizer.param_groups[0]['lr'] = global_optimizer.param_groups[0]['lr']    

    for local_model in local_models:
       local_model.load_state_dict(global_model.state_dict())
    sync_end_time = time.time()
    print('*'*50)
    print(f'Time taken for synchronization: {str(timedelta(seconds=sync_end_time - sync_start_time))}')
    test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
    print(f'Local Step {iteration+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
    print('*'*50)
    
# TODO: Schedule ro check konim 

  total_end_time = time.time()
  print('/'*50)
  print(f'Total time taken for local_SGD: {str(timedelta(seconds=total_end_time - total_start_time))}')
  print('/'*50)


In [ ]:
lr = 1e-02
wd = 1e-03
beta = 0.4
alpha = 1
parameters = {'lr': lr, 'wd': wd, 'beta': beta}
K = [2, 4, 8]
J = [4, 8, 16, 32 , 64]
num_epochs = 150
data = CIFAR100Data()
loss_fn = nn.CrossEntropyLoss()

# Initialize slow model
slow_model = LeNet5()
slow_model.load_state_dict(initial_state_dict)

for k in K:
  shard_loaders = data.iid_shards(num_shards=k)
  for j in J:
    print('='*50)
    print(f'Number of Workers:{k}, Number of Local Steps:{j}')
    print('='*50)
    local_SGD_SlowMo(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs, beta, alpha)

Files already downloaded and verified
Files already downloaded and verified
Number of Workers:2, Number of Local Steps:1
Worker 1, [01/01]: Training Loss: 4.342150847, Training Accuracy: 3.840
Time taken for training worker 1: 0:00:16.096309
--------------------------------------------------
Worker 2, [01/01]: Training Loss: 4.335856533, Training Accuracy: 4.268
Time taken for training worker 2: 0:00:13.447262
--------------------------------------------------
**************************************************
Time taken for synchronization: 0:00:00.002601
Local Step 01: Test Loss: 3.988315181, Test Accuracy: 8.490
**************************************************
Worker 1, [01/01]: Training Loss: 3.885550671, Training Accuracy: 9.632
Time taken for training worker 1: 0:00:13.360038
--------------------------------------------------
Worker 2, [01/01]: Training Loss: 3.893093649, Training Accuracy: 9.688
Time taken for training worker 2: 0:00:11.400283
---------------------------------

# **Personal Contribution**

### SHAT (Asyncronous Approach)

In [62]:
import random
import math
from torch.optim import Optimizer

class SHAT_PS_Optimizer(Optimizer):
    def __init__(self, global_model, lr=0.01):
        self.global_model = global_model
        self.lr = lr
        params = list(global_model.parameters())
        super(SHAT_PS_Optimizer, self).__init__(params, {'lr': lr})
        
    def step(self, gradients_model):
        # Update global model parameters with gradients 
        with torch.no_grad():
            for key, param in self.global_model.state_dict().items():
                param.copy_(param -  self.lr * gradients_model.state_dict()[key])

        return None

# For example total iterations 150*8 = 1200 to hacve the same number of iterations as local SGD in other words 150 true epochs
# For example: Worker 1 has 15 iterations, Worker 2 has 15 iterations, Worker 3 has 15 iterations, Worker 4 has 15 iterations
# All have the same number of iterations

# TODO: In this situation we will NOT have any bias to the worker that has higher iterations but the time that it takes is equal to the time of slowest worker that would be equal to local SGD
def generate_computation_latency_sequence(K, each_worker_iteration):
    """
    Simulates a sequence of operations for K workers with random computation latencies.

    The function generates a list of operations, each performed by a worker, sorted by
    their latencies. Each worker performs exactly t operations, and the latencies are
    scaled so that the fastest worker's latency is normalized to 1.

    Parameters:
    K (int): The number of workers in the simulation.
    t (int): The number of operations each computer will perform.

    Returns:
    tuple: A tuple containing:
        - result (list of dicts): A list of dictionaries where each dictionary represents
          an operation performed by a computer. The keys in the dictionary are:
              - "total_iterations" (int): The total number of iterations performed.
              - "computer" (int): The ID of the computer performing the operation.
              - "value" (int): The latency value associated with that operation.
        - computation_powers (list of ints): The original random computation latencies for each computer.
        - scaled_powers (list of floats): The computation latencies scaled such that the fastest
          worker's latency is 1.
    """
    # Generate random computation powers
    computation_powers = [random.randint(5000, 10000) for _ in range(K)]

    # Find the minimum computation power
    min_power = min(computation_powers)

    # Scale the computation powers so that the fastest (smallest value) is equal to 1
    scaled_powers = [power / min_power for power in computation_powers]

    # Initialize a list to store the dictionaries
    result = []

    # Store the number of turns taken by each worker
    turns_taken = [0] * K
    total_number_of_iterations = 0
    # Generate the dictionaries until all computers have 15 turns
    while any(turn < each_worker_iteration for turn in turns_taken):
        total_number_of_iterations += 1
        # Generate the next possible dictionaries for each worker
        possible_entries = []
        for index in range(K):
            if turns_taken[index] < each_worker_iteration:  # Only consider computers with less than 15 turns
                operation_value = computation_powers[index] * (turns_taken[index] + 1)
                possible_entries.append({"total_iterations": total_number_of_iterations,"worker": index, "value": operation_value})

        # Sort the possible dictionaries by their operation value
        possible_entries.sort(key=lambda x: x["value"])

        # Add the dictionary with the smallest value to the result
        chosen_entry = possible_entries[0]
        result.append(chosen_entry)

        # Update the turn count for the chosen worker
        chosen_index = chosen_entry["worker"] 
        turns_taken[chosen_index] += 1

    return result, computation_powers, scaled_powers


# For example total iterations 150*8 = 1200 to have the same number of iterations as local SGD in other words 150 true epochs
# For example: Worker 1 has 15 iterations, Worker 2 has 30 iterations, Worker 3 has 45 iterations, Worker 4 has 60 iterations
# TODO: in this situation we will have bias to ther worker that has higher iterations
'''def generate_computation_latency_sequence(K, total_iterations):
    """
    Simulates a sequence of operations for K workers with random computation latencies.

    The function generates a list of operations, each performed by a worker, sorted by
    their latencies. The total number of iterations is limited by `total_iterations`,
    and the latencies are scaled so that the fastest worker's latency is normalized to 1.

    Parameters:
    K (int): The number of workers in the simulation.
    total_iterations (int): The total number of iterations across all workers.

    Returns:
    tuple: A tuple containing:
        - result (list of dicts): A list of dictionaries where each dictionary represents
          an operation performed by a worker. The keys in the dictionary are:
              - "total_iterations" (int): The total number of iterations performed.
              - "worker" (int): The ID of the worker performing the operation.
              - "value" (int): The latency value associated with that operation.
        - computation_powers (list of ints): The original random computation latencies for each worker.
        - scaled_powers (list of floats): The computation latencies scaled such that the fastest
          worker's latency is 1.
    """
    # Generate random computation powers
    computation_powers = [random.randint(5000, 10000) for _ in range(K)]

    # Find the minimum computation power
    min_power = min(computation_powers)

    # Scale the computation powers so that the fastest (smallest value) is equal to 1
    scaled_powers = [power / min_power for power in computation_powers]

    # Initialize a list to store the dictionaries
    result = []

    # Store the number of turns taken by each worker
    turns_taken = [0] * K
    total_number_of_iterations_performed = 0

    # Generate the dictionaries until the total number of iterations is reached
    while total_number_of_iterations_performed < total_iterations:
        total_number_of_iterations_performed += 1

        # Generate the next possible dictionaries for each worker
        possible_entries = []
        for index in range(K):
            # Calculate the operation value based on the current number of turns for this worker
            operation_value = computation_powers[index] * (turns_taken[index] + 1)
            possible_entries.append({
                "total_iterations": total_number_of_iterations_performed,
                "worker": index,  # Now starting index from 0
                "value": operation_value
            })

        # Sort the possible dictionaries by their operation value
        possible_entries.sort(key=lambda x: x["value"])

        # Add the dictionary with the smallest value to the result
        chosen_entry = possible_entries[0]
        result.append(chosen_entry)

        # Update the turn count for the chosen worker
        chosen_index = chosen_entry["worker"]
        turns_taken[chosen_index] += 1

    return result, computation_powers, scaled_powers
'''

def calculate_gradients_model(global_model, local_model, lr):
    gradients_dict = {key: torch.zeros_like(value) for key, value in local_model.state_dict().items()}
    
    for key, value in local_model.state_dict().items():
        gradients_dict[key] += ((global_model.state_dict()[key] - value)/lr)
    # TODO: lr ro check konim 
    
    return gradients_dict

def update_global_model(global_model, gradients_model, lr):
    new_weights = {}
    for key, value in global_model.state_dict().items():
        new_weights[key] = value - lr * gradients_model.state_dict()[key]

    return new_weights


def calculate_s_i(k,ci):
    return float(k/ci)

def calculate_alpha_i(si, k):
    alpha = 1 - (si / math.log(k))
    return alpha

def updatel_local_model(global_model, local_model, alpha_i):
    new_weights = {}
    for key, value in local_model.state_dict().items():
        new_weights[key] = (1 - alpha_i) * value + alpha_i * global_model.state_dict()[key]
    
    return new_weights



In [17]:
import time
from datetime import timedelta

def SHAT_PS(shard_loaders, loss_fn, k, lr, wd, initial_state_dict, num_epochs):  
    # SHAT Parameter Server to manage workers
    total_start_time = time.time()

    # Initialize a model with same value of param for each chunk
    local_models = [LeNet5().to(device) for _ in range(k)]
    for model in local_models:
      model.load_state_dict(initial_state_dict)

    # # Initialize the global model
    global_model = LeNet5().to(device)
    global_model.load_state_dict(initial_state_dict)

    global_optimizer = SHAT_PS_Optimizer(global_model, lr)
  
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(global_optimizer, T_max=num_epochs)
    # Generate a sequence of computation latency to simulate the difference of computation latency (Lower computation Latency means higher computation power)
    computation_latency_sequence, computation_latency, scaled_computation_latency = generate_computation_latency_sequence(k, num_epochs)

    # Print the original and scaled computation latency
    print("Original Computation Latency:", computation_latency)
    print("Scaled Computation Latency:", scaled_computation_latency)

    # print sequence of workers based on their computation latency
    print(f'workers simulated orders based on computation latency:{[entry["worker"]+1 for entry in computation_latency_sequence]}')

    C = [1 for _ in range(k)] # Staleness counter
    local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd) for model in local_models]
    
    #This specifies turn of the model
    for iteration_index, worker in enumerate([entry['worker'] for entry in computation_latency_sequence]):
      iteration_start_time = time.time()
      print('*'*50)

      train_loss, train_accuracy = train(local_models[worker], shard_loaders[worker], local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
      print(f'Worker {worker+1}, [{iteration_index+1:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      print('*'*50)
      
      '''PS server: receive model from the worker and calculate diff model (gradient)'''
      gradients_model = LeNet5().to(device)
      gradients_model.load_state_dict(calculate_gradients_model(global_model, local_models[worker], lr))
      
      # Computing the staleness of each worker
      for i in C :
        if i != worker:
          i += 1
          
      '''PS Server update global model'''
      global_optimizer.step(gradients_model)
      
      '''send updated model to the worker'''
      '''calucale the staleness of the worker αi ← si − logn ,    s ← n/ci'''
      s_i = calculate_s_i(k,C[worker])
      alpha_i = calculate_alpha_i(s_i, k)
      '''update worker local model ba w ← (1 − α )w + α w̃'''
      local_models[worker].load_state_dict(updatel_local_model(global_model, local_models[worker], alpha_i))

      '''continue outer loop in PS'''
      '''ci = 0 ya 1'''
      C[worker] = 1
      
      iteration_end_time = time.time()
     
      print(f'Time taken for worker {worker+1} : {str(timedelta(seconds=iteration_end_time - iteration_start_time))}')
      print('-'*50)
      print(f'Iteration: {iteration_index+1:02}/{k*num_epochs:02} - True epochs: {num_epochs}')
      test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
      print(f'Local Step {iteration_index+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
      print('-'*50)

    total_end_time = time.time()
    print('/'*50)
    print(f'Total time taken for SHAT: {str(timedelta(seconds=total_end_time - total_start_time))}')
    print('/'*50)
      

In [18]:
# TODO: Baraye tedade local step shak daram
# SHAT
lr = 1e-02 # Khodesh gofte hamina ro baraye CIFAR 10 bezarim vali dar morede CIFAR 100 chizi nagorfte
wd = 1e-03 
K = [2, 4, 8] # TODO: Nemidoonam chand bezaram
num_epochs = 150
data = CIFAR100Data()
loss_fn = nn.CrossEntropyLoss()

for k in K: # number of workers
  shard_loaders = data.iid_shards(num_shards=k)
  print('='*50)
  print(f'Number of Workers:{k}')
  print('='*50)
  SHAT_PS(shard_loaders, loss_fn, k, lr, wd, initial_state_dict, num_epochs)



Files already downloaded and verified
Files already downloaded and verified
Number of Workers:2
Original Computation Latency: [8401, 7812]
Scaled Computation Latency: [1.0753968253968254, 1.0]
workers simulated orders based on computation latency:[2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1,

KeyboardInterrupt: 

### LocalAdaScale (Choice of LR in LocalSGD)

In [27]:
# def synchronize(models):
#   for params in zip(*[model.parameters() for model in models]):
#     param_avg = torch.mean(torch.stack([param.data for param in params]), dim=0)
#     for param in params:
#       param.data = param_avg
  
#   return models[0]

# def synchronize(global_model, local_models):
#         # Initialize a state dict with zeros, same shape as the model parameters
#         delta = {key: torch.zeros_like(value) for key, value in local_models[0].state_dict().items()}
      
#         # Sum up all the model parameters
#         for local_model in local_models:
#             for key, value in local_model.state_dict().items():
#                 delta[key] += (global_model.state_dict()[key] - value)
      
#         # Divide each parameter by the number of models to get the average
#         for key in delta:
#             delta[key] /= len(local_models)
      
#         new_weights = {}
#         for key, value in global_model.state_dict().items():
#             new_weights[key] = value -  delta [key] # TODO: az TA beporsim ke learning rate ro chetor hesab konim.
      
#         global_model.load_state_dict(new_weights)
#         return global_model

def average_models(local_models):
    """Calculate the average model from a list of local models."""
    num_models = len(local_models)
    
    # Initialize the averaged model as a copy of the first local model
    avg_model = local_models[0]
    
    # Zero the parameters of the avg_model to start accumulating
    for param in avg_model.parameters():
        param.data.zero_()
    
    # Accumulate the parameters from all local models
    for local_model in local_models:
        for avg_param, local_param in zip(avg_model.parameters(), local_model.parameters()):
            avg_param.data.add_(local_param.data)
    
    # Divide by the number of models to compute the average
    for param in avg_model.parameters():
        param.data.div_(num_models)
    
    return avg_model

In [14]:
import time
from datetime import timedelta

def LocalAdaScale(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs):
  total_start_time = time.time()

  iterations = num_epochs // j
  # Initialize a model with same value of param for each chunk
  local_models = [LeNet5().to(device) for _ in range(k)]
  for model in local_models:
    model.load_state_dict(initial_state_dict)
  #Initialize optimizers for each chunk
  local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd) for model in local_models]
  # global_optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
  # Initialize a scheduler for each optimizer
  # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(global_optimizer, T_max=iterations)
  # Initialize the global model
  global_model = LeNet5().to(device)
  global_model.load_state_dict(initial_state_dict)


  for iteration in range(iterations):
    for worker, shard_loader in enumerate(shard_loaders):
      train_start_time = time.time()
      for loca_step in range(j):
        train_loss, train_accuracy = train(local_models[worker], shard_loader, local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
        print(f'Worker {worker+1}, [{loca_step+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      train_end_time = time.time()
      print(f'Time taken for training worker {worker+1}: {str(timedelta(seconds=train_end_time - train_start_time))}')
      print('-'*50)
    sync_start_time = time.time()

    # global_model = synchronize(global_model, local_models, 1)
    global_model = synchronize(local_models)

    # for local_model in local_models:
    #   local_model.load_state_dict(global_model.state_dict())
    # scheduler.step()
    sync_end_time = time.time()
    print('*'*50)
    print(f'Time taken for synchronization: {str(timedelta(seconds=sync_end_time - sync_start_time))}')
    test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
    print(f'Local Step {iteration+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
    print('*'*50)
    
# TODO: Schedule ro check konim 

  total_end_time = time.time()
  print('/'*50)
  print(f'Total time taken for local_SGD: {str(timedelta(seconds=total_end_time - total_start_time))}')
  print('/'*50)

In [15]:
# import torch
# (data_loader, loss_fn,  num_workers, scale_inv_budget, lr_init, wd,initial_state_dict, num_epochs)
# def local_adascale_local_sgd(data_loader, num_epochs, lr_init, num_workers, num_local_steps, scale_inv_budget):
#     # Initialize a model and save its initial parameters
#     model = LeNet5().to(device='cuda').load_state_dict(initial_state_dict)
    
#     optimizer = torch.optim.SGD(model.parameters(), lr=lr_init)
    
#     # Add a learning rate scheduler
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
#     grad_cache = [None] * num_workers
#     scale_inv_counter = 0
    
#     for epoch in range(num_epochs):
#         for i, (inputs, targets) in enumerate(data_loader):
#             # Simulate local steps on different workers
#             for worker_id in range(num_workers):
#                 outputs = model(inputs)
#                 loss = torch.nn.functional.cross_entropy(outputs, targets)
#                 optimizer.zero_grad()
#                 loss.backward()
                
#                 if (i + 1) % num_local_steps == 0:
#                     grad_cache[worker_id] = [param.grad.clone() for param in model.parameters()]
                    
#             # Synchronize and average gradients after num_local_steps
#             if (i + 1) % num_local_steps == 0:
#                 avg_grad = [torch.mean(torch.stack([grad_cache[worker_id][j] for worker_id in range(num_workers)]), dim=0)
#                             for j in range(len(grad_cache[0]))]
                
#                 # Compute gradient statistics
#                 grad_variance = [torch.var(torch.stack([grad_cache[worker_id][j] for worker_id in range(num_workers)]), dim=0)
#                                  for j in range(len(grad_cache[0]))]
#                 grad_mean = [torch.mean(avg_grad[j]) for j in range(len(avg_grad))]
                
#                 gain_ratio = compute_gain_ratio(grad_mean, grad_variance, num_workers, num_local_steps)
                
#                 # Apply scaled gradients
#                 for param, grad in zip(model.parameters(), avg_grad):
#                     param.grad = gain_ratio * grad
                
#                 # Scale invariant iteration counter
#                 scale_inv_counter += gain_ratio
                
#                 # Update model parameters
#                 optimizer.step()

#             # Check if scale invariant budget is exhausted
#             if scale_inv_counter >= scale_inv_budget:
#                 break
        
#         # Step the scheduler at the end of each epoch
#         scheduler.step()
    
#     return model

# def compute_gain_ratio(grad_mean, grad_variance, num_workers, num_local_steps):
#     gain_ratio = []
#     for g_mean, g_var in zip(grad_mean, grad_variance):
#         g2_mean = g_mean ** 2
#         term1 = g2_mean + g_var / num_workers
#         term2 = (g2_mean + g_var / num_workers) ** 2
#         term3 = 3 * (num_local_steps - 1) * g2_mean * g_var
#         gain_ratio.append(2 * term1 / (term1 + torch.sqrt(term2 + term3)))
    
#     return torch.tensor(gain_ratio).mean().item()

# Example usage:
# model = MyModel()
# data_loader = DataLoader(my_dataset, batch_size=64, shuffle=True)
# trained_model = local_adascale_local_sgd(model, data_loader, num_epochs=10, lr_init=0.1, num_workers=4, num_local_steps=5, scale_inv_budget=1000)


In [28]:
from torch.optim import Optimizer
    
class CustomSGDLocalAdaScaleOptimizer(Optimizer):
    def __init__(self, params, lr=0.01, gain_ratio=1.0, momentum=0.9, weight_decay=0.0):
        # Store default settings
        defaults = dict(lr=lr, gain_ratio=gain_ratio, momentum=momentum, weight_decay=weight_decay)
        super(CustomSGDLocalAdaScaleOptimizer, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            lr = group['lr']
            gain_ratio = group['gain_ratio']
            momentum = group['momentum']
            weight_decay = group['weight_decay']

            for param in group['params']:
                if param.grad is None:
                    continue

                d_p = param.grad.data

                # Apply weight decay if specified
                if weight_decay != 0:
                    d_p.add_(param.data, alpha=weight_decay)

                # If momentum is used
                if momentum != 0:
                    if 'momentum_buffer' not in self.state[param]:
                        buf = self.state[param]['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = self.state[param]['momentum_buffer']
                        buf.mul_(momentum).add_(d_p)
                    d_p = buf

                # Custom update rule
                try:
                    param.data.add_(d_p, alpha=-gain_ratio * lr)
                except:
                    pass

        return loss

In [40]:
import time
from datetime import timedelta
import math
def local_adascale(shard_loaders, loss_fn, K, j, lr, wd, initial_state_dict, scale_invariant_budget):
    total_start_time = time.time()
    global_model = LeNet5().to(device)
    global_model.load_state_dict(initial_state_dict)
    local_models = [LeNet5().to(device) for _ in range(K)]
    for model in local_models:
      model.load_state_dict(initial_state_dict)
    # local_optimizers = [CustomOptimizer(model.parameters(), lr=0.01, gain_ratio=1) for model in local_models]
    # local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9) for model in local_models]
    local_optimizers = [CustomSGDLocalAdaScaleOptimizer(model.parameters(), lr, gain_ratio=1, momentum=0.9, weight_decay= wd) for model in local_models]


    num_epochs =150
    scale_invariant_budget = 150
    # for s in scale_invariant_budget: # Bejaye scale_invariant_budget, num_epochs ro dar nazar gereftim vali goftim age s>scale_invariant_budget shod break kon.
    scale_inv_counter = 0
    gain_ratio = 1
    for epoch in range(num_epochs):
        grad_cache = []
        for hi, h in enumerate(range(j)):
            for k in range(K):
                # calculate gradient of all workers g^t_k
                train_loss, train_accuracy, gradient =local_update_with_gradient(local_models[k], local_optimizers[k], shard_loaders[k], loss_fn, lr, gain_ratio)
                print(f'Worker {k+1}, [{h+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
                if hi == 0: # Only store the gradients for the first local step
                    grad_cache.append(gradient)
                

            # ------------
            
            # varriance bar va G bar bayad adad bashan va faghat tooye synchronization point (after H steps mohasebe mishan)
            # fek konam in mishe g bar_t
            # list ba len=10 (ehtemalan tedade layer ha) va dakhele harkodoom 2 ta tensor ke ehtemalen weights and bias
            # TA Inja Kar mikone
            # TODO: variance bar, G bar, rho, lr tebghe paper dar zamane syncronization update mishan.
        
        global_model = average_models(local_models)
        for local_model in local_models:
            local_model.load_state_dict(global_model.state_dict())
        # To check if averaging is done correctly (for only two workers)
        # with torch.no_grad():
        #     for param1, param2, avg_param in zip(local_models[0].parameters(), local_models[1].parameters(), average_model.parameters()):
        #         calculated_avg = (param1.data + param2.data) / 2
        #         assert torch.allclose(avg_param.data, calculated_avg), "Averaging failed!"
        
        # test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
        # print(f'Local Step {epoch+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
        print(50*'-')
        g_bar = calculate_g_bar(grad_cache, K)
        variance_bar = calculate_variance_bar(g_bar, grad_cache, K)
        G_bar = calculate_G_bar(g_bar=g_bar, num_workers=K, variance_bar=variance_bar)
        L = estimate_lipschitz_constant(global_model, shard_loaders[0], loss_fn,device='cuda')
        gain_ratio_new = compute_gain_ratio(G_bar,variance_bar, K, j)
        lr_new = compute_optimal_learning_rate(G_bar,variance_bar, K, j, L)
        if not (lr_new < 0 or math.isnan(lr_new) or gain_ratio_new < 0 or math.isnan(gain_ratio_new)):
        # lr = lr_new
            gain_ratio = gain_ratio_new
        print(f'variance_bar: {variance_bar}, G_bar: {G_bar}, gain_ratio: {gain_ratio}, L: {L}, lr: {lr}')
        # Scale invariant iteration counter
        scale_inv_counter += gain_ratio
        if scale_inv_counter > scale_invariant_budget:
            break 
        print (f'scale_inv_counter: {scale_inv_counter} : scale_invariant_budget: {scale_invariant_budget}')
        print (f'gain_ratio: {gain_ratio} lr: {lr}')

    
    total_end_time = time.time()
    print('/'*50)
    print(f'Total time taken for LocalAdaScale: {str(timedelta(seconds=total_end_time - total_start_time))}')
    print('/'*50)

def calculate_g_bar(grad_cache, num_workers):
    """Calculate ḡ: Element-wise average of all gradients in grad_cache."""
    gbar = [torch.mean(torch.stack([grad_cache[worker_id][i] for worker_id in range(num_workers)]), dim=0)
            for i in range(len(grad_cache[0]))]
    
    return gbar


'''def compute_gain_ratio(grad_mean, grad_variance, num_workers, num_local_steps):
    gain_ratio = []
    for g_mean, g_var in zip(grad_mean, grad_variance):
        g2_mean = g_mean ** 2
        term1 = g2_mean + g_var / num_workers
        term2 = (g2_mean + g_var / num_workers) ** 2
        term3 = 3 * (num_local_steps - 1) * g2_mean * g_var
        gain_ratio.append(2 * term1 / (term1 + torch.sqrt(term2 + term3)))
    
    return torch.tensor(gain_ratio).mean().item()'''

def compute_gain_ratio(G_bar, variance_bar, num_workers, num_local_steps):
    # Use Ḡ and variancē to compute the gain ratio
    term0 = 2 * (G_bar + variance_bar)
    term1 = G_bar + variance_bar / num_workers
    term2 = term1 ** 2
    term3 = 3 * (num_local_steps - 1) * G_bar * variance_bar
    
    gain_ratio = term0 / (term1 + torch.sqrt(term2 + term3))
    # print(f'variance_bar/G_bar: {variance_bar/G_bar}') # Vaghti variance_bar/G_bar miad taghriban zire 10 hame chi be ham mirize va gain ratio nan mishe.
    if torch.isnan(gain_ratio):
        pass
        pass
    # Ke variance_bar/G_bar tabe'e tedade worker e, va shayad tedade iteration

        # time.sleep(2)

    return gain_ratio.item()

def compute_optimal_learning_rate(G_bar, variance_bar, num_workers, num_local_steps, L):
    # Use Ḡ and variancē to compute the adaptive learning rate (t)
    term0 = 2 * (G_bar)
    term1 = G_bar + variance_bar / num_workers
    term2 = term1 ** 2
    term3 = 3 * (num_local_steps - 1) * G_bar * variance_bar
    
    lr = term0 / (term1 + torch.sqrt(term2 + term3))
    lr = lr / L
    return lr.item()

def calculate_G_bar(g_bar, variance_bar, num_workers):
    """Calculate Ḡ as a scalar."""
    g_bar_norm = sum(torch.norm(g_bar[i])**2 for i in range(len(g_bar)))
    G_bar = g_bar_norm - (1 / num_workers) * variance_bar
    # print(f'G_bar: {G_bar}')
    # print(G_bar.item())
    if G_bar.item() < 0:
        print('Gbaaaaaaaaaaaaar is negative')
        # G_bar = 0.00
    #     G_bar = 0.02* variance_bar # Choon bazi vaghta adad mishe kamtar az sefr choon taghrib mizanim. vali in kari ke kardam ham dorost nist.
    #     print(f'heeeeeeeey: g_bar_norm: {g_bar_norm}, variance_bar: {variance_bar}, (1 / num_workers) * variance_bar: {(1 / num_workers) * variance_bar}')
    #     print('-')

    return G_bar

def calculate_variance_bar(g_bar, grad_cache, num_workers): # Hamishe mosbate.
    """Calculate variancē as a scalar."""
    sum_norms = sum(torch.norm(grad_cache[worker_id][i])**2 for worker_id in range(num_workers) for i in range(len(g_bar)))
    nomrs_sum = 0
    for worker_id in range(num_workers):
        for i in range(len(g_bar)):
            nomrs_sum(torch.norm(grad_cache[worker_id][i])**2)
    g_bar_norm = sum(torch.norm(g_bar[i])**2 for i in range(len(g_bar)))
    variance_bar = (1 / (num_workers - 1)) * sum_norms - (num_workers / (num_workers - 1)) * g_bar_norm
    print(f'sum(norm(g^t_k)**2): {sum_norms}, g_bar_norm: {g_bar_norm}')
    return variance_bar

def local_update_with_gradient(model, optimizer, dataloader, loss_fn, lr, gain_ratio, device=device, is_wandb=False):# (X, y, model, criterion, learning_rate): # forward_backward_pass_manual
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Update optimizer parameters
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        param_group['gain_ratio'] = gain_ratio
        
        
    
    ## model.zero_grad()  # Initialize gradients to zero at the start of each epoch
    optimizer.zero_grad()  # Reset gradients
    
    initial_params = [param.clone() for param in model.parameters()] 

    for i, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        loss.backward()  # Backpropagation

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        optimizer.step() 
        if i == (len(dataloader) - 1):
            gradient = [param.grad.clone() for param in model.parameters()] 
            
        optimizer.zero_grad()

    final_params = [param.clone() for param in model.parameters()]  # Capture final parameters

    # Calculate delta as the difference between final and initial parameters
    # gradient = [(final - initial)/(gain_ratio * lr) for final, initial in zip(final_params, initial_params)]
    
    # TODO: in faghat gradient e akhar ro hesab mikone  behtare ke 
    #gradient = [param.grad.clone() for param in model.parameters()] 
    
    # Update parameters using custom rule
    # with torch.no_grad():  # Disable gradient tracking for manual update
    #     for param in model.parameters():
    #         if param.grad is not None:
    #             param -= gain_ratio * lr * param.grad

    # model.zero_grad()
    len_dataloader = len(dataloader)
    train_loss = running_loss / len_dataloader
    train_accuracy = 100. * correct / total
    if math.isnan(train_loss) or train_loss > 10:
        pass
    if is_wandb:
        wandb.log({"Train Loss": train_loss, "Train Accuracy": train_accuracy})

    return train_loss, train_accuracy, gradient

def estimate_lipschitz_constant(model, dataloader, loss_fn, device='cuda'):
    model.eval()  # Set the model to evaluation mode
    max_grad_norm = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        model.zero_grad()
        loss.backward()  # Compute gradients
        
        # Compute the gradient norm
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                param_norm = p.grad.data.norm(2)
                total_norm += param_norm.item() ** 2
        
        total_norm = total_norm ** 0.5
        max_grad_norm = max(max_grad_norm, total_norm)
    
    # Estimate L as the maximum observed gradient norm
    lipschitz_constant = max_grad_norm
    return lipschitz_constant


In [41]:
lr = 1e-02
wd = 1e-03
K = [8, 2, 4, 8]
J = [2, 2, 4, 8, 16, 32, 64] 
scale_invariant_budget = 150 # It uses as a unit instead of number of epochs
data = CIFAR100Data()
loss_fn = nn.CrossEntropyLoss()

for k in K: # Number of workers
  shard_loaders = data.iid_shards(num_shards=k)
  for j in J:
    print('='*50)
    print(f'Number of Workers:{k}, Number of Local Steps:{j}')
    print('='*50)
    # (, loss_fn, , j, lr, wd, initial_state_dict, num_epochs)           
    # local_adascale_local_sgd(shard_loaders, loss_fn,  k, scale_inv_budget, lr, wd,initial_state_dict, num_epochs)
    local_adascale(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, scale_invariant_budget)
    # LocalAdaScale(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs)

Files already downloaded and verified
Files already downloaded and verified
Number of Workers:8, Number of Local Steps:2
Worker 1, [01/02]: Training Loss: 4.585575668, Training Accuracy: 1.888
Worker 2, [01/02]: Training Loss: 4.587634257, Training Accuracy: 1.920
Worker 3, [01/02]: Training Loss: 4.590183788, Training Accuracy: 1.088
Worker 4, [01/02]: Training Loss: 4.585954588, Training Accuracy: 1.760
Worker 5, [01/02]: Training Loss: 4.590105193, Training Accuracy: 1.456
Worker 6, [01/02]: Training Loss: 4.593763317, Training Accuracy: 1.296
Worker 7, [01/02]: Training Loss: 4.591303373, Training Accuracy: 1.584
Worker 8, [01/02]: Training Loss: 4.588327033, Training Accuracy: 1.648
Worker 1, [02/02]: Training Loss: 4.349892193, Training Accuracy: 4.400
Worker 2, [02/02]: Training Loss: 4.383812447, Training Accuracy: 3.632
Worker 3, [02/02]: Training Loss: 4.370596905, Training Accuracy: 4.336
Worker 4, [02/02]: Training Loss: 4.384550151, Training Accuracy: 3.760
Worker 5, [02/0

TypeError: 'int' object is not callable

### Our Approach 1: HeteroCompSGD

In [13]:
# def lock_layers(model, ratio):
#     """
#     Locks the first `ratio` portion of the layers in the model.

#     Parameters:
#     - model (nn.Module): The model whose layers you want to lock.
#     - ratio (float): The ratio of layers to lock. For example, 0.25 will lock the first 25% of layers.
#     """
#     # Count the number of layers in the model
#     layers = list(model.children())
#     num_layers = len(layers)
    
#     # Calculate the number of layers to lock
#     layers_to_lock = int(ratio * num_layers)
#     if ratio * num_layers != layers_to_lock:  # If not an exact integer, round up
#         layers_to_lock += 1
    
#     # Lock the first `layers_to_lock` layers
#     count = 0
#     for layer in model.children():
#         if count < layers_to_lock:
#             for param in layer.parameters():
#                 param.requires_grad = False
#             count += 1
#         else:
#             break

def lock_conv_and_fc_layers(model, ratio):
    """
    Locks the first `ratio` portion of the Conv2d and Linear layers in the model.

    Parameters:
    - model (nn.Module): The model whose layers you want to lock.
    - ratio (float): The ratio of Conv2d and Linear layers to lock. For example, 0.25 will lock the first 25% of such layers.
    """
    # Flatten the model layers into a list (recursively)
    layers = []
    def get_layers(module):
        for layer in module.children():
            if isinstance(layer, (nn.Conv2d, nn.Linear)):
                layers.append(layer)
            if len(list(layer.children())) > 0:  # If the layer has sub-layers
                get_layers(layer)
    
    get_layers(model)
    num_layers = len(layers)
    
    # Calculate the number of layers to lock
    layers_to_lock = int(ratio * num_layers)
    if ratio * num_layers != layers_to_lock:  # If not an exact integer, round up
        layers_to_lock += 1
    
    # Lock the first `layers_to_lock` Conv2d and Linear layers
    count = 0
    for layer in layers:
        if count < layers_to_lock:
            for param in layer.parameters():
                param.requires_grad = False
            count += 1
        else:
            break

def synchronize(models): # average models over only unlocked parameters
    for params in zip(*[model.parameters() for model in models]):
        # Filter out the parameters that are locked (i.e., requires_grad is False)
        unlocked_params = [param.data for param in params if param.requires_grad]
        
        # If there are any unlocked parameters, average them
        if unlocked_params:
            param_avg = torch.mean(torch.stack(unlocked_params), dim=0)
            for param in params:
                if param.requires_grad:  # Update only the unlocked parameters
                    param.data = param_avg
    
    return models[0]


In [17]:
import time
from datetime import timedelta

def HeteroCompSGD(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs, warmups):
  total_start_time = time.time()
  print('Start Time:', time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(total_start_time)))
  iterations = num_epochs // j
  # Initialize a model with same value of param for each chunk
  local_models = [LeNet5().to(device) for _ in range(k)]
  for model in local_models:
    model.load_state_dict(initial_state_dict)
  #Initialize optimizers for each chunk
  local_optimizers = [torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd) for model in local_models]
  # global_optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
  # Initialize a scheduler for each optimizer
  # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(global_optimizer, T_max=iterations)
  # Initialize the global model
  global_model = LeNet5().to(device)
  global_model.load_state_dict(initial_state_dict)

  workers_warmup_time = {}
  print('-'*50)
  print('Warmup rounds')
  print('-'*50)

  for warmup in range(warmups):
    for worker, shard_loader in enumerate(shard_loaders):
      train_start_time = time.time()
      for local_step in range(j):
        train_loss, train_accuracy = train(local_models[worker], shard_loader, local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
        print(f'Worker {worker+1}, [{local_step+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      train_end_time = time.time()
      time_diff = train_end_time - train_start_time
      
      if worker == 1:
        time_diff = workers_warmup_time[0] * 1.25
      workers_warmup_time[worker] = time_diff      
      print(f'Time taken for warmup {worker+1}: {str(timedelta(seconds=train_end_time - train_start_time))}')
      print('-'*50)

  print(f'workers_warmup_time: {workers_warmup_time}')
  print('Warmup finished')
  print('-'*50)

  min_time = min(workers_warmup_time.values())
  workers_warmup_time_ratio = {worker:  (1-(min_time/time)) for worker, time in workers_warmup_time.items()}
  print(f'scaled_workers_warmup_time: {workers_warmup_time_ratio}')

  for worker, local_model in enumerate(local_models):
    lock_conv_and_fc_layers(local_model, workers_warmup_time_ratio[worker])
  
  for iteration in range(iterations):
    for worker, shard_loader in enumerate(shard_loaders):
      train_start_time = time.time()
      for local_step in range(j):
        train_loss, train_accuracy = train(local_models[worker], shard_loader, local_optimizers[worker], loss_fn, device = device,  is_wandb=False)
        print(f'Worker {worker+1}, [{local_step+1:02}/{j:02}]: Training Loss: {train_loss:.9f}, Training Accuracy: {train_accuracy:.3f}')
      train_end_time = time.time()
      print(f'Time taken for training worker {worker+1}: {str(timedelta(seconds=train_end_time - train_start_time))}')
      print('-'*50)
    sync_start_time = time.time()

    global_model = synchronize(local_models)

    for local_model in local_models:
       local_model.load_state_dict(global_model.state_dict())
    # scheduler.step()
    sync_end_time = time.time()
    print('*'*50)
    print(f'Time taken for synchronization: {str(timedelta(seconds=sync_end_time - sync_start_time))}')
    test_loss, test_accuracy = test(global_model,original_test_loader, loss_fn, is_wandb = False)
    print(f'Local Step {iteration+1:02}: Test Loss: {test_loss:.9f}, Test Accuracy: {test_accuracy:.3f}')
    print('*'*50)
    
# TODO: Schedule ro check konim 

  total_end_time = time.time()
  print('/'*50)
  print(f'Total time taken for HeteroCompSGD: {str(timedelta(seconds=total_end_time - total_start_time))}')
  print('/'*50)

In [18]:
lr = 1e-02
wd = 1e-03
K = [2, 4]
J = [2, 4, 8, 16, 32, 64] 
num_epochs = 150
warmups = 1
data = CIFAR100Data()
loss_fn = nn.CrossEntropyLoss()

for k in K: # Number of workers
  shard_loaders = data.iid_shards(num_shards=k)
  for j in J:
    print('='*50)
    print(f'Number of Workers:{k}, Number of Local Steps:{j}')
    print('='*50)
    HeteroCompSGD(shard_loaders, loss_fn, k, j, lr, wd, initial_state_dict, num_epochs, warmups)

Files already downloaded and verified
Files already downloaded and verified
Number of Workers:2, Number of Local Steps:2
Start Time: 2024-09-02 17:26:05
Worker 1, [01/02]: Training Loss: 4.293673834, Training Accuracy: 4.732
Worker 1, [02/02]: Training Loss: 3.844355908, Training Accuracy: 11.016
Time taken for warmup 1: 0:00:23.047949
--------------------------------------------------
Worker 2, [01/02]: Training Loss: 4.301435818, Training Accuracy: 4.568
Worker 2, [02/02]: Training Loss: 3.837007832, Training Accuracy: 10.452
Time taken for warmup 2: 0:00:24.374378
--------------------------------------------------
workers_warmup_time: {0: 23.04794931411743, 1: 28.80993664264679}
scaled_workers_warmup_time: {0: 0.0, 1: 0.19999999999999996}
Worker 1, [01/02]: Training Loss: 3.602551771, Training Accuracy: 14.420
Worker 1, [02/02]: Training Loss: 3.394790099, Training Accuracy: 17.648
Time taken for training worker 1: 0:00:24.614722
--------------------------------------------------
Wo

KeyboardInterrupt: 